 <font size="6">Обучение на реальных данных</font>

# Проблемы при работе с реальной задачей машинного обучения

В реальных задачах, особенно если вы работаете над новой проблемой, вы столкнетесь с широким спектром трудностей. Приведем часть из них и сопроводим примером на основе задач нахождения клеток крови на фотографии мазка крови.

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L11/bloods_cell.jpg" width="500px">

- **нехватка данных** — фотографий мазков крови может быть недостаточно для построения сложной модели с нуля

- **недостаток размеченных данных** — возможно, существует достаточно большое количество фотографий мазков крови (например, в историях болезни), но очень малая часть из них размечена

- **некачественная разметка** — мазок крови могли доверить анализировать студенту-практиканту. Размечать его мог вообще человек не из профессии — например, хотевший таким образом увеличить обучающую выборку для модели на конкурс Kaggle. Даже в широко известных MNIST, CIFAR-10 и ImageNet есть ошибки в разметке ([примеры](https://labelerrors.com/))

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L11/wrong_imagenet_prediction.png" width="300">

<em>Source: <a href="https://labelerrors.com/">labelerrors.com</a></em>

- **низкое качество данных** — не все фотографии будут в хорошем разрешении. Да и не все образцы были правильно подготовлены. А еще могут попадаться фотографии от пациентов, больных чем-нибудь экзотическим, в результате чего их клетки выглядят сильно отличающимися от обычных. Или же в крови пациента просто плавают паразиты

Серповидная клеточная анемия приводит к аномальным эритроцитам

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L11/normal_and_sickle_shaped_erythrocytes.JPG" width="500px">

<em>Source: <a href="https://commons.wikimedia.org/wiki/File:Sicklecell3.jpg">Wikipedia</a></em>

А так выглядит мазок крови при сонной болезни

<img src="https://edunet.kea.su/repo/EduNet-web_dependencies/L11/trypanosoma_among_red_blood_cells.jpg" width="500px">

<em>Source: <a href="https://commons.wikimedia.org/wiki/File:Trypanosoma_sp._PHIL_613_lores.jpg">Wikipedia</a></em> 

- **несбалансированность датасета** — клетки крови встречаются в разных пропорциях. Какие-то классы могут быть плохо представлены (**минорные классы**) Потому в вашем датасете может быть всего 10 фотографий, на которых присутствуют базофилы. Нейросети будет очень заманчиво вообще не пытаться найти базофилы (всего 10 ошибок).

<img src="https://edunet.kea.su/repo/EduNet-content/L11/out/leukocyte_blood_formula_table.png" >



- **ковариантный сдвиг** — явление, когда признаки тренировочной и тестовой выборок **распределены по-разному**. Ковариантный сдвиг может стать серьезной проблемой для практического применения моделей.

Модель учится сопоставлять целевые значения признакам. В такой ситуации модель не в состоянии делать адекватные предсказания на тесте, так как во время обучения она не видела области пространства, в которой расположены тестовые объекты.
Источники ошибок, приводящих к ковариантному сдвигу, обсуждались ранее [в лекции №7](https://edunet.kea.su/repo/msu_ai/html/L07_Batch_normalization.html#Covariate-shift-(Ковариантный-сдвиг)).

**Практический совет:** для быстрого обнаружения ковариантного сдвига можно **обучить модель**, которая будет предсказывать, относится ли объект к **train** или **test** выборке. Если модель легко делит данные, то имеет смысл визуализировать значения признаков, по которым она это делает.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/covariate_shift.png" width="600">

- **полные дубликаты** — в данных могут быть полные дубликаты. Кто-то до вас агрегировал фотографии из разных источников, и вы либо не обратили на это внимание, либо он забыл об этом сказать. Такие данные надо сразу помечать и использовать только после предварительного размышления, т.к. они могут мешать вам и на этапе обучения модели, и на итоговой валидации ее качества (если один и тот же объект попадет и в обучение, и в валидацию). 

- **неполные дубликаты** — в данных могут быть данные от одного и того же пациента. Кажется, что если это разные мазки крови, то всё нормально. На самом деле и это уменьшает количество информации, которую может извлечь нейросеть из данных. С такими данными также нужно аккуратно работать и не допускать попадания одного пациента и в обучение, и в тест.

- **малое число источников данных** — проблема, родственная предыдущей. В вашем датасете могут быть данные только от одного микроскопа или одной модели микроскопа. Могут быть данные, снятые только одним специалистом, или в одной больнице, или только у взрослых (фотографий мазков детей нет). Это так же может влиять на способность вашего алгоритма генерализовывать полученное решение и требует пристального внимания.

Все это приводит к целому спектру проблем, из которых самой типичной будет переобучение модели — какую бы простую модель вы не взяли, она все равно будет выучивать искажения вашего датасета. 

# Общие подходы при работе с реальными данными


## Нехватка данных 

Если у вас мало данных, попробуйте найти еще данные для вашей задачи. 
Совет приводится во многих инструкциях по борьбе с малым количеством данных и может быть воспринят с юмором. 
Однако часто для вашей задачи действительно существуют данные, собранные другими людьми. Также часто можно найти данные, которые очень похожи на ваши, и их можно использовать в обучении, но, например, учитывать с меньшим весом.
Даже 20 дополнительных примеров могут сильно облегчить ситуацию. 

Вы также можете использовать данные, которые не совсем похожи на ваши, в качестве внешней валидации. Тем самым вы можете разбивать свой изначальный датасет только для кросс-валидации и не выделять отдельную часть для теста. Тестом послужат как раз "не совсем" похожие данные. 

Также можно использовать две техники, которые мы сегодня рассмотрим подробнее: Аугментация и Transfer Learning.

## Дисбаланс классов

Прежде всего надо убедиться, что датасет сбалансирован:

In [ ]:
import torch
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine


def show_class_balance(y, classes):
    _, counts = torch.unique(torch.tensor(y), return_counts=True)
    plt.bar(classes, counts)
    plt.ylabel("n_samples")
    plt.ylim([0, 75])
    plt.show()


wine = load_wine()
classes = wine.target_names

show_class_balance(wine.target, classes)

Разница в 10-20% будет незначительна, поэтому для наглядности мы искусственно разбалансируем наш датасет при помощи метода [make_imbalance](https://imbalanced-learn.org/stable/references/generated/imblearn.datasets.make_imbalance.html)

In [ ]:
from imblearn.datasets import make_imbalance

x, y = make_imbalance(
    wine.data, wine.target, sampling_strategy={0: 10, 1: 70, 2: 40}, random_state=42
)
show_class_balance(y, classes)

### Изменение баланса класса сэмплированием 

Если в данных недостаток именно конкретного класса, то можно бороться с этим при помощи разных способов сэмплирования. 

Важно понимать, что в большинстве случаев данные, полученные таким способом, должны использоваться в качестве **обучающего набора**, но ни в коем случае не в качестве **валидации** или **теста**. 


#### Дублирование примеров меньшего класса (oversampling)

Мы можем увеличить число объектов меньшего класса за счет дублирования. 

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/oversampling_scheme.png" width="600"></center>

<center><em>Дублирование примеров меньшего класса</em></center>



В этом случае наша модель будет "вынуждена" обращать внимание на минорный класс. 

Такой `Resampling` может быть выполнен с помощью пакета [imbalanced-learn](https://pypi.org/project/imbalanced-learn/), как показано ниже:

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)
x_ros, y_ros = ros.fit_resample(x, y)

show_class_balance(y_ros, classes)

#### Уменьшение числа примеров большего класса (undersampling)

Аналогично, можно взять для обучения не всех представителей большего класса. 

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/undersampling_scheme.png" width="600"></center>

<center><em>Удаление примеров преобладающего класса</em></center>

Это также вынуждает модель обращать внимание на оба класса. 
Минус подхода очевиден: мы можем выбросить важных представителей большего класса, ответственных за существенное улучшение генерализации,  и за счет этого качество модели существенно ухудшится. 
Можно пытаться выбрасывать объекты большего класса как-то по-умному. Например, кластеризовать объекты большего класса и брать по заданному количеству объектов из каждого класса. 

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
x_res, y_res = rus.fit_resample(x, y)

show_class_balance(y_res, classes)

#### Ансамбли + undersampling

Можно использовать ансамбли вместе с undersampling. В этом случае мы можем, к примеру, делать сэмплирование только большего класса, а объекты минорного класса оставлять как есть.

<img src="https://edunet.kea.su/repo/EduNet-content/L11/out/ensembles_and_undersampling.png" width="700px">

Или просто сэмплировать объектов и того, и другого класса равное количество.

#### Балансирование представленности объектов в батчах

В случае нейросетей можно балансировать встречаемость каждого класса не на уровне датасета, а на уровне батча. Например, собираем каждый батч таким образом, чтобы в нем было поровну всех классов.

 Это может улучшать сходимость даже в случае небольшого дисбаланса или его отсутствия, т.к. мы будем избегать шаги обучения нейросети, в которых она просто не увидела какого-то класса в силу случайных причин.


<img src="https://edunet.kea.su/repo/EduNet-content/L11/out/batch_balancing.png">

В PyTorch эту функциональность можно получить, используя класс [WeightedRandomSampler](https://pytorch.org/docs/stable/data.html#torch.utils.data.WeightedRandomSampler) . Для его инициализации требуется рассчитать вес каждого класса. Сумма весов не обязана быть равной единице.

In [ ]:
# https://pytorch.org/docs/stable/generated/torch.unique.html
_, counts = torch.unique(torch.tensor(y), return_counts=True)
weights = counts.max() / counts
print("Classes: ", classes)
print("Weights: ", weights)

Теперь создаем объект [WeightedRandomSampler](https://pytorch.org/docs/stable/data.html#torch.utils.data.WeightedRandomSampler), в конструктор подаем два аргумента:

- список весов для **каждого** элемента в датасете;
- количество элементов (можно использовать не весь датасет).

In [ ]:
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler

tensor_x = torch.Tensor(x)  # transform to torch tensor
tensor_y = torch.Tensor(y)
dataset = TensorDataset(tensor_x, tensor_y)

batch_size = 8

weight_for_sample = []  # Every sample must have a weight
for l in y:
    weight_for_sample.append(weights[l].item())

sampler = WeightedRandomSampler(torch.tensor(weight_for_sample), len(dataset))
loader = DataLoader(dataset, batch_size=32, drop_last=True, sampler=sampler)

Посмотрим на распределение элементов разных классов по батчам.

In [ ]:
batch_labels = []
for data, labels in loader:
    print(
        "Labels:",
        labels.int().tolist(),
        "Classes in batch:",
        torch.unique(labels, return_counts=True)[1].tolist(),
    )
    batch_labels.append(labels.tolist())

show_class_balance(batch_labels, classes)

Результаты будут отличаться от запуска к запуску. Но видно, что в батчах объекты каждого класса встречаются почти равномерно.

Стоит отметить, что нужно быть осторожным со взвешиванием объектов в батчах и **контролировать состав батчей**. Дело в том, что при существенном дисбалансе веса при объектах минорного класса могут оказываться на несколько порядков больше, чем при объектах мажорного класса. Данные веса преобразуются в вероятности для сэмплирования, и может случиться так, что вероятности при объектах мажорного класса станут численно неотличимы от нуля. Тем самым можно получить обратный эффект: батчи будут состоять исключительно из объектов минорного класса. В таком случае нужно намеренно ограничивать веса, чтобы не допускать такого эффекта.

### Генерация синтетических данных

Другой подход к решению этой проблемы — создание синтетических данных. Делать это можно по-разному. 



#### SMOTE

**Synthetic Minority Over-sampling Technique (SMOTE)** позволяет генерировать синтетические данные за счет реальных объектов из минорного класса. 

Алгоритм работает следующим образом:

1. Для случайной точки из минорного класса выбираем $k$ ближайших соседей из того же класса. 
2. Для первого соседа проводим отрезок, соединяющий его и выбранную точку. На этом отрезке случайно выбираем точку. 
3. Эта точка — новый **синтетический** объект минорного класса.
4. Повторяем процедуру для оставшихся соседей.


<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/generate_synthetic_data.png" width="700">

Число соседей, как и число раз, которое мы запускаем описанную выше процедуру, можно регулировать.

In [ ]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
x_smote, y_smote = oversample.fit_resample(x, y)

show_class_balance(y_smote, classes)

Количество объектов каждого класса, которое должно получиться после генерации, можно задать явно:

In [ ]:
over = SMOTE(sampling_strategy={0: 20, 1: 70, 2: 70})
x_smote, y_smote = over.fit_resample(x, y)

show_class_balance(y_smote, classes)

Подробнее про использование пакета можно прочесть в статье: [SMOTE for Imbalanced Classification with Python](https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/)

### Изменение функции потерь

Модели в машинном обучении “ленивы”. При работе с несбалансированными классами модель будет чаще сталкиваться с доминирующим классом и вместо того, чтобы разбираться в признаках объектов, может начать ориентироваться на статистическое распределение классов. 

Пример: датасет, в котором 95% объектов относятся к классу 1 и 5% к классу 0. Модель может выучиться всегда относить объекты к классу 1, и в 95% случаях она будет права.

#### Веса классов

Чтобы это поправить, можно изменить функцию потерь так, чтобы больше штрафовать модель за ошибки в минорных классах. Для этого можно:
- Добавлять веса в функцию потерь для компенсации дисбаланса классов.
Во многих функциях потерь в PyTorch (например, [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)) есть параметр `weight`, который имеет по умолчанию значение `None`. В него можно передать тензор весов, соответствующий размеру вектора целевых значений, и получить взвешенную функцию ошибок.

Посмотрим, как это работает. Допустим, мы получили от нейросети неверные предсказания: 

второй объект должен относиться к классу 1, а не 0

In [ ]:
scores = torch.tensor([[30.0, 2.0], [30.0, 2.0]])  # Scores for batch of two samples
target = torch.tensor([0, 1])  # Second sample belongs to class 1
# but logit for class 0 greater: 30 > 2. So it was misclassified

Подсчитаем Cross-Entropy Loss без весов:

$$Loss =  \frac{1}{2} \biggr[- \ln\frac{e^{30}}{e^{30}+e^{2}} - \ln\frac{e^{2}}{e^{30}+e^{2}}\biggr]\approx 14.0 $$

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(scores, target)
print(f"Loss = {loss.item():.2f}")

Если у нас есть два класса с соотношением 4:1, можно задать веса `weight = [0.2, 0.8]`. И так как сеть ошиблась на классе с большим весом, то ошибка вырастет

$$Loss =  \biggr[-0.2 \ln\frac{e^{30}}{e^{30}+e^{2}} -0.8 \ln\frac{e^{2}}{e^{30}+e^{2}}\biggr]\approx 22.4 $$

In [ ]:
weights = torch.tensor([0.2, 0.8], dtype=torch.float32)
criterion = torch.nn.CrossEntropyLoss(weight=weights)
loss = criterion(scores, target)
print(f"Loss = {loss.item():.2f}")

Сумма весов может быть не равна единице:

In [ ]:
criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor([2.0, 8.0]))
loss = criterion(scores, target)
print(f"Loss = {loss.item():.2f}")

- Иногда качество модели можно улучшить, взяв квадратные корни от полученных таким образом весов (немного снижает штрафы за ошибки на редких классах).

- Не смотря на интуитивно понятную логику работы способа, он не всегда дает значительный эффект. Тем не менее, на практике стоит пробовать экспериментировать с этим способом наряду с прочими техниками борьбы с дисбалансом.

- Можно менять форму функции потерь. В 2017 году для работы с несбалансированными классами был предложен [Focal Loss](https://arxiv.org/abs/1708.02002?context=cs). Это — модификация Cross-Entropy Loss, которая модифицирует ее [форму](https://medium.com/visionwizard/understanding-focal-loss-a-quick-read-b914422913e7) для различных классов. С тех пор появились различные [модификации](https://paperswithcode.com/method/focal-loss) функции ошибок, с которыми можно и нужно экспериментировать. 

####  Focal Loss

Focal Loss — это функция потерь, используемая в нейронных сетях для решения проблемы классификации *сложных* объектов (hard examples).

**Приведем пример:** пусть модель должна классифицировать фрукты на два класса: яблоки и груши. В наборе данных есть много явных представителей того и иного класса: зеленые яблоки и желтые груши. Для модели они будут *простыми*: она на них не ошибается, и предсказываемая вероятность истинного класса для них велика и равна $0.9$. 

В то же время в наборе данных есть малое количество зеленых груш: они по форме похожи на груши, а по цвету — на яблоки. Говоря более общими словами, зеленые груши *ближе к границе классов в пространстве признаков*. Для модели такие примеры могут оказаться *сложными*, и она будет на них ошибаться. Пусть для зеленой груши вероятность истинного класса равна $0.2$, то есть модель ошиблась и предсказала класс "яблоко" с вероятностью $0.8$.

<img src='https://edunet.kea.su/repo/EduNet-content/L11/out/hard_examples_fruits.png' width=800></img>

Проблема состоит в том, что сумма большого количества малых ошибок на *простых* объектах может перевешивать сумму малого количества ошибок потерь на *сложных* объектах. Поэтому модель будет плохо учиться верно классифицировать сложные объекты: ей будет "лень" исправлять незначительные ошибки.

Для решения этой проблемы была предложена специальная функция потерь — Focal Loss. Она немного модифицирует кросс-энтропию для придания большей значимости ошибкам на сложных объектах.

Focal Loss была предложена в статье [Focal Loss for Dense Object Detection (Lin et al., 2017)](https://arxiv.org/abs/1708.02002) изначально для задачи детектирования объектов на изображениях. Она определяется так:

$$\large\text{FL}(p_t) = -(1 - p_t)^\gamma\text{log}(p_t)$$

Здесь $p_t$ — предсказанная вероятность истинного класса, а $\gamma$ — настраиваемый гиперпараметр. 

Focal Loss уменьшает потери на уверенно классифицируемых примерах (где $p_t>0.5$) и больше фокусируется на сложных примерах, которые классифицированы неправильно. Параметр $\gamma$ управляет относительной важностью неправильно классифицируемых примеров. Более высокое значение $\gamma$ увеличивает важность неправильно классифицированных примеров. В экспериментах авторы показали, что параметр $\gamma=2$ показывал себя наилучшим образом в их задаче.

При $\gamma=0$ Focal Loss становится равной Cross-Entropy Loss, которая выражается как обратный логарифм вероятности истинного класса:

$$\large\text{CE}(p_t)=-\text{log}(p_t)$$

**Разберем на нашем примере** с яблоками и грушами. 

Мы имеем **20 простых** объектов с вероятностью истинного класса $0.9$: 10 яблок и 10 груш, **и один сложный** объект с вероятностью истинного класса $0.2$.


$\large{\text{CE} = \overbrace{\sum^{20}-\text{log}(0.9)}^{\large\color{#3C8031}{\text{loss(easy apples and pears)=2.11}}} + \overbrace{(-\text{log}(0.2))}^{\large\color{#F26035}{\text{loss(hard pear)=1.61}}} \approx 3.72}$

$\large{\text{FL}(\gamma=2) = \overbrace{\sum^{20}-\color{#AF3235}{\underbrace{(1-0.9)^2}_{0.01}}\text{log}(0.9)}^{\large\color{#3C8031}{\text{loss(easy apples and pears)=0.02}}} + \overbrace{(-\color{#AF3235}{\underbrace{(1-0.2)^2}_{0.64}}\text{log}(0.2))}^{\large\color{#F26035}{\text{loss(hard pear)=1.03}}} \approx 1.05}$

Фактически, потери для уверенно классифицированных объектов дополнительно занижаются.

Этот эффект достигается путем домножения на коэффициент: $ \large(1-p_{t})^\gamma$

Пока модель ошибается, $p_{t}$ — мала, и значение выражения в скобках соответственно близко к 1.

Когда модель обучилась, значение $p_{t}$ становится близким к 1, а разность в скобках становится маленьким числом, которое возводится в степень $ \gamma \ge 0 $. Таким образом, домножение на это небольшое число нивелирует вклад верно классифицированных объектов.

Это позволяет модели сосредоточиться (сфокусироваться, отсюда и название) на изучении сложных объектов (hard examples).





В примере коэффициент $(1-p_t)^\gamma$ в Focal Loss в 100 раз занизил потери при уверенной классификации простых яблок и груш, и потери при неверной классификации сложной груши стали преобладать.

<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L11/focal_loss_vs_ce.png" width="700"></center>

<center><em>Source: <a href="https://arxiv.org/abs/1708.02002">Focal Loss for Dense Object Detection (Lin et al., 2018)</a></em></center>



Давайте посчитаем для различных значений $γ$, сколько понадобится примеров с небольшой ошибкой (высокой вероятностью истинного класса, равной $0.9$), чтобы получить суммарный **Focal Loss** примерно такой же, как у одного примера с большой ошибкой (низкой вероятностью истинного класса, равной $0.2$).

In [ ]:
import numpy as np

def cross_entropy(prob_true):
    return -np.log(prob_true)


def focal_loss(prob_true, gamma=2):
    return (1 - prob_true) ** gamma * cross_entropy(prob_true)


p1 = 0.9  # probability of easy examples predictions
p2 = 0.2  # probability of hard examples predictions
gammas = [0, 0.5, 1, 2, 5, 10, 15]

print(
    f"For probability of easy examples predictions {p1} and probability of hard examples predictions {p2}\n"
)

for gamma in gammas:
    fl1 = focal_loss(p1, gamma)
    fl2 = focal_loss(p2, gamma)

    print(
        f"gamma = {gamma},".ljust(15),
        f"for an equal loss with a problematic prediction, almost correct ones are required {int(fl2 / fl1)}",
    )

Как видно, при увеличении значения $\gamma$ можно достичь значительного роста "важности" примеров с высокой ошибкой, что по сути позволяет модели обращать внимание на "hard examples".

Этот пример также показывает **опасность Focal Loss**: если мы имеем **ошибки в разметке**, то при большом $\gamma$ можно начать очень сильно наказывать модель за ошибки на неверно размеченных примерах, что может привести к переобучению под ошибки в разметке.

Focal Loss может применяться также и в задачах с дисбалансом классов. В этом смысле объекты преобладающего класса могут считаться простыми, а объекты минорного класса — сложными.

Однако для работы с дисбалансом в Focal Loss могут быть добавлены веса для классов. Тогда формула будет выглядеть так:

$$\large\text{FL}(p_t) = -\alpha_t(1 - p_t)^\gamma\text{log}(p_t)$$

Здесь $\alpha_t$ — вес для истинного класса, имеющий такой же смысл, как параметр `weight` в Cross-Entropy Loss.

Focal Loss не реализована в PyTorch нативно, но существуют сторонние совместимые реализации. Посмотрим, как воспользоваться [одной из них](https://github.com/AdeelH/pytorch-multi-class-focal-loss).

In [ ]:
import random

def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)


set_random_seed(42)

In [ ]:
from IPython.display import clear_output

!wget https://raw.githubusercontent.com/AdeelH/pytorch-multi-class-focal-loss/master/focal_loss.py
clear_output()

In [ ]:
import torch
from torch import nn
from focal_loss import FocalLoss


criterion = FocalLoss(alpha=None, gamma=2.)

model_output = torch.rand(3, 3)  # model output is logits, as in CELoss
print(f"model_output:\n {model_output}")

target = torch.empty(3, dtype=torch.long).random_(3)
print(f"target: {target}")

loss_fl = criterion(model_output, target)
print(f"loss_fl: {loss_fl}")

Убедимся, что сторонняя реализация вычисляет то, что нужно, и вычислим значение вручную. В первую очередь нужно перевести `model_output` из логитов в вероятности с помощью softmax.

In [ ]:
probs = torch.nn.functional.softmax(model_output, dim=1)

print(f"probabilities after softmax:\n {probs}")

Теперь вручную рассчитаем значение функции потерь.

In [ ]:
def focal_loss(prob_true, gamma=2):
    return - (1 - prob_true) ** gamma * np.log(prob_true)

hand_calculated_loss = 0

for i in range(3):
    hand_calculated_loss += focal_loss(probs[i, target[i]])

hand_calculated_loss /= 3  # average by number of samples

print(f"hand-calculated focal loss:    {hand_calculated_loss.item()}")
print(f"library-calculated focal loss: {loss_fl}")
print(f"Are results almost equal? {torch.isclose(loss_fl, hand_calculated_loss).item()}")

Действительно, при расчете вручную получили то же значение, что и при расчете с помощью сторонней реализации.

### Обнаружение аномалий / изменений

В случае сильно несбалансированных наборов данных, таких как fraud (мошенничество) или машинный сбой, стоит задуматься, могут ли такие примеры рассматриваться как аномалия (выброс) или нет. Если такое событие и впрямь может считаться аномальным, мы можем использовать такие модели, как `OneClassSVM`, методы кластеризации или методы обнаружения гауссовских аномалий.

Эти методы требуют изменения взгляда на задачу: мы будем рассматривать аномалии как отдельный класс выбросов. Это может помочь нам найти новые способы разделения и классификации.

Если продолжать пример с фруктами, то задача обнаружения аномалий возникла бы, если бы мы предполагали, что среди яблок и груш может вдруг возникнуть мандарин, или любой другой фрукт, и нам бы нужно было не отнести его к одному из известных классов, а пометить как отдельный, отличающийся класс.

<img src='https://edunet.kea.su/repo/EduNet-content/L11/out/anomaly_fruits.png' width='800'></img>

Проблемой при работе с аномалиями является то, что аномальных значений может быть очень мало или вообще не быть. В таком случае алгоритм учит паттерны нормального поведения и реагирует на отличия от паттернов.

Разберем примеры обнаружения аномалий с помощью трех алгоритмов из библиотеки Scikit-Learn (там можно найти еще много различных алгоритмов).

Создадим датасет из двух кластеров и случайных значений.

In [ ]:
import numpy as np

rng = np.random.RandomState(42)

# Train
x = 0.3 * rng.randn(100, 2)  # 100 2D points
x_train = np.r_[x + 2, x - 2]  # split into two clusters

# Test norlmal
x = 0.3 * rng.randn(20, 2)  # 20 2D points
x_test_norlmal = np.r_[x + 2, x - 2]  # split into two clusters

# Test outliers
x_test_outliers = rng.uniform(low=-4, high=4, size=(20, 2))

Напишем функцию визуализации, которая будет изображать созданный датасет на рисунке слева, а результат поиска аномалий — на рисунке справа.

In [ ]:
def plot_outliers(x_train, x_test_norlmal, x_test_outliers, model=None):
    fig, (plt_data, plt_model) = plt.subplots(1, 2, figsize=(12, 6))

    plt_data.set_title("Created Dataset (real labels)")
    plot_train = plt_data.scatter(
        x_train[:, 0], x_train[:, 1], c="white", s=40, edgecolor="k"
    )
    plot_test_normal = plt_data.scatter(
        x_test_norlmal[:, 0], x_test_norlmal[:, 1], c="green", s=40, edgecolor="k"
    )
    plot_test_outliers = plt_data.scatter(
        x_test_outliers[:, 0], x_test_outliers[:, 1], c="red", s=40, edgecolor="k"
    )

    plt_data.set_xlim((-5, 5))
    plt_data.set_ylim((-5, 5))

    plt_data.legend(
        [plot_train, plot_test_normal, plot_test_outliers],
        ["train", "test normal", "test outliers"],
        loc="lower right",
    )

    if model:
        plt_model.set_title("Model Results")
        # plot decision function
        xx, yy = np.meshgrid(np.linspace(-5, 5, 50), np.linspace(-5, 5, 50))
        Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        plt_model.contourf(xx, yy, Z, cmap=plt.cm.Blues_r)

        # plot prediction
        full_data = np.concatenate((x_train, x_test_norlmal, x_test_outliers), axis=0)
        predicted = model.predict(full_data)

        anom_index = np.where(predicted == -1)
        anom_values = full_data[anom_index]

        plot_all_data = plt_model.scatter(
            full_data[:, 0], full_data[:, 1], c="white", s=40, edgecolor="k"
        )

        plot_anom_data = plt_model.scatter(
            anom_values[:, 0], anom_values[:, 1], c="red", s=40, marker="x"
        )
        plt_model.legend(
            [plot_all_data, plot_anom_data],
            ["normal", "outliers"],
            loc="lower right",
        )
    plt.show()

Посмотрим, как работает на этих данных алгоритм [OneClassSVM](https://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html).

Идея алгоритма состоит в поиске функции, которая положительна для областей с высокой плотностью и отрицательна для областей с малой плотностью. Подробнее об алгоритме можно прочитать в оригинальной [статье](https://proceedings.neurips.cc/paper/1999/file/8725fb777f25776ffa9076e44fcfd776-Paper.pdf).



In [ ]:
from sklearn.svm import OneClassSVM

gamma = 2.0
contamination = 0.05

model = OneClassSVM(gamma=gamma, kernel="rbf", nu=contamination)
model.fit(x_train)

plot_outliers(x_train, x_test_norlmal, x_test_outliers, model)

Посмотрим, как на этих же данных работает алгоритм [IsolationForest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html). 

IsolationForest состоит из деревьев, которые «изолируют» (пытаются отделить от остальной выборки) наблюдения, случайным образом выбирая признак и случайное значение порога для этого признака (между max и min значениями признака). Такой алгоритм чаще и проще отделяет значения аномалии. Если построить по такому принципу множество деревьев, то значения, которые чаще других отделяются раньше, будут аномалиями.


In [ ]:
from sklearn.ensemble import IsolationForest

n_estimators = 200
contamination = 0.05

model = IsolationForest(
    n_estimators=n_estimators, contamination=contamination, random_state=rng
)
model.fit(x_train)

plot_outliers(x_train, x_test_norlmal, x_test_outliers, model)

Последним алгоритмом, на который мы посмотрим, будет [LocalOutlierFactor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.LocalOutlierFactor.html).

В нем используется метод k-NN. Расстояние до ближайших соседей используется для оценки расположения точек. Если соседи далеко, то точка с большой вероятностью является аномалией. 

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

n_neighbors = 10
contamination = 0.05

model = LocalOutlierFactor(
    n_neighbors=n_neighbors, novelty=True, contamination=contamination
)
model.fit(x_train)

plot_outliers(x_train, x_test_norlmal, x_test_outliers, model)

### Метрики качества на несбалансированных данных

Обращайте внимание на то, какие метрики вы используете. При решении задачи классификации часто используется accuracy (точность), равная доле правильно классифицированных объектов. Эта метрика позволяет адекватно оценить результат классификации в случае сбалансированных классов. В случае дисбаланса классов данная метрика может выдать обманчиво хороший результат.

Пример: датасет, в котором 95% объектов относятся к классу 0 и 5% к классу 1. 

In [ ]:
from sklearn.datasets import make_classification
from collections import Counter


x, y = make_classification(
    n_samples=1000,
    n_features=2,
    n_redundant=0,
    n_clusters_per_class=1,
    weights=[0.95],
    flip_y=0,
    random_state=42,
)

counter = Counter(y)
print("Class distribution ", Counter(y))

for label, _ in counter.items():
    row_ix = np.where(y == label)[0]
    plt.scatter(x[row_ix, 0], x[row_ix, 1], label=str(label))
plt.legend()
plt.show()

И модель, которая для всех данных выдает класс 0, 

In [ ]:
class DummyModel:
    def predict(self, x):
        return np.zeros(x.shape[0])  # always predict class 0

Такая модель будет иметь $accuracy = 0.95$, хотя не выдает никакой полезной информации:



In [ ]:
from sklearn.metrics import accuracy_score

dummy_model = DummyModel()
y_pred = dummy_model.predict(x)

accuracy = accuracy_score(y, y_pred)
print("Accuracy", accuracy)

Для несбалансированных данных лучше выбирать [F1 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html?highlight=f1_score#sklearn.metrics.f1_score),  [MCC](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html?highlight=matthews%20correlation%20coefficient#sklearn.metrics.matthews_corrcoef) (Matthews correlation coefficient, коэффициент корреляции Мэтьюса) или [balanced accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html#sklearn.metrics.balanced_accuracy_score) (среднее между recall разных классов).

In [ ]:
from sklearn.metrics import f1_score, matthews_corrcoef, balanced_accuracy_score

print("F1", f1_score(y, y_pred))
print("MCC", matthews_corrcoef(y, y_pred))
print("Balanced accuracy", balanced_accuracy_score(y, y_pred))

Все эти метрики оказываются равны нулю, что отражает отсутствие связи предсказаний с данными на входе модели.

Дополнительно: [Your Dataset Is Imbalanced? Do Nothing!](https://towardsdatascience.com/your-dataset-is-imbalanced-do-nothing-abf6a0049813)

## Аугментация

Другой способ побороть маленькое количество данных для обучения — аугментация.

**Аугмента́ция** (от лат. augmentatio — увеличение, расширение) — увеличение выборки обучающих данных через модификацию существующих данных.

Модели глубокого обучения обычно требуют большого количества данных для обучения. В целом, чем больше данных, тем лучше для обучения модели. В то же время получение огромных объемов данных сопряжено со своими проблемами (например, с нехваткой размеченных данных или с трудозатратами, сопряженными с разметкой).

Вместо того, чтобы тратить дни на сбор данных вручную, мы можем использовать методы аугментации для автоматической генерации новых примеров из уже имеющихся.

Помимо увеличения размеченных датасетов, многие методы *self-supervised learning* построены на использовании разных аугментаций одного и того же сэмпла.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/augmentations_examples.png" width="700"></center>
<center><em>Примеры аугментаций картинки. </em></center>

**Важный момент**: при обучении модели мы используем разбиение данных на `train-val-test`. Аугментации стоит применять только на `train`. Почему так? Конечная цель обучения нейросети — это применение на реальных данных, которые сеть не видела. Поэтому для адекватной оценки качества модели, валидационные и тестовые данные изменять не нужно.

В любом случае, `test` должен быть отделен от данных еще до того, как они попали в `DataLoader` или нейросеть.

Другое дело, что аугментации на тесте можно использовать как метод ансамблирования в случае классификации. Можно взять sample → создать несколько его копий → по-разному их аугментировать → предсказать класс на каждой из этих аугментированных копий → а потом выбрать наиболее вероятный класс голосованием (такой функционал реализован, например, в [YOLOv5](https://github.com/ultralytics/yolov5/blob/d204a61834d0f6b2e73c1f43facf32fbadb6b284/models/yolo.py#L121), о которой речь пойдет в следующих лекциях).

### Изображения

Загрузим и отобразим пример картинки. Картинку отмасштабируем, чтобы она не занимала весь экран.

In [ ]:
# fix random_seed
set_random_seed(42)

# Compute on cpu or gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

URL = "https://edunet.kea.su/repo/EduNet-web_dependencies/L11/capybara_image.jpg"
!wget -q $URL -O test.jpg

In [ ]:
from IPython.display import display
from PIL import Image
from torchvision import transforms

input_img = Image.open("/content/test.jpg")
input_img = transforms.Resize(size=300)(input_img)
display(input_img)

Рассмотрим несколько примеров аугментаций картинок. С полным списком можно ознакомиться на сайте [[doc] документации torchvision](https://pytorch.org/vision/stable/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py).

#### Random Rotation

Трансформация [`transforms.Random Rotation`](https://pytorch.org/vision/main/generated/torchvision.transforms.RandomRotation.html?highlight=randomrotation#torchvision.transforms.RandomRotation) принимает параметр `degrees` — диапазон углов, из которого выбирается случайный угол для поворота изображения.

Создадим переменную `transform`, в которую добавим нашу аугментацию, и применим ее к исходному изображению. Затем запустим следующую ячейку несколько раз подряд

In [ ]:
import matplotlib.pyplot as plt


def plot_augmented_img(transform, input_img):
    f, ax = plt.subplots(1, 2, figsize=(15, 15))
    augmented_img = transform(input_img)
    ax[0].imshow(input_img)
    ax[0].set_title("Original img")
    ax[0].axis("off")

    ax[1].imshow(augmented_img)
    ax[1].set_title("Augmented img")
    ax[1].axis("off")
    plt.show()


transform = transforms.RandomRotation(degrees=(0, 180))

plot_augmented_img(transform, input_img)

#### Gaussian Blur
[`transforms.GaussianBlur`](https://pytorch.org/vision/main/generated/torchvision.transforms.GaussianBlur.html?highlight=gaussianblur#torchvision.transforms.GaussianBlur) размывает изображение с помощью фильтра Гаусса.

In [ ]:
transform = transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))

plot_augmented_img(transform, input_img)

#### Random Erasing
[`transforms.RandomErasing`](https://pytorch.org/vision/main/generated/torchvision.transforms.RandomErasing.html?highlight=transforms+randomerasing#torchvision.transforms.RandomErasing) стирает на изображении произвольный прямоугольник. Она имеет параметр `p` — вероятность, с которой данная трансформация вообще применится к изображению.

Данная трансформация работает только с `torch.Tensor`, поэтому предварительно нужно применить трансформацию `ToTensor`, а затем `ToPILImage`, чтобы воспользоваться нашей функцией для отображения.



In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.RandomErasing(p=1), transforms.ToPILImage()]
)

plot_augmented_img(transform, input_img)

Не лишним будет заметить, что некоторые трансформации могут существенно исказить изображение. Например, здесь, `RandomErasing` практически полностью стерла основной объект на снимке — капибару. Такая грубая аугментация может только навредить процессу обучения, и на практике нужно быть осторожным.

`RandomErasing` также имеет параметр `scale` — диапазон соотношения стираемой области к входному изображению. Попробуем уменьшить этот диапазон относительно значения по умолчанию, чтобы избежать нежелательного эффекта стирания капибары.

In [ ]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.RandomErasing(p=1, scale=(0.02, 0.1)),
        transforms.ToPILImage(),
    ]
)

plot_augmented_img(transform, input_img)

#### ColorJitter
[`transforms.ColorJitter`](https://pytorch.org/vision/main/generated/torchvision.transforms.ColorJitter.html?highlight=colorjitter#torchvision.transforms.ColorJitter) случайным образом меняет яркость, контрастность, насыщенность и оттенок изображения.

In [ ]:
transform = transforms.ColorJitter(brightness=0.5, hue=0.3)

plot_augmented_img(transform, input_img)

#### Совмещаем несколько аугментаций вместе

Для этого будем использовать метод [`transforms.Compose`](https://pytorch.org/vision/stable/generated/torchvision.transforms.Compose.html?highlight=compose#torchvision.transforms.Compose). Нам нужно будет создать `list` со всеми аугментациями, которые будут применены последовательно.

In [ ]:
transform = transforms.Compose(
    [
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.RandomPerspective(distortion_scale=0.5, p=1.0),
        transforms.ColorJitter(brightness=0.5, hue=0.3),
    ]
)

plot_augmented_img(transform, input_img)

#### Совмещение нескольких аугментаций случайным образом


##### <font size="4">Random Apply</font>

А что если мы хотим применять аугментации случайным образом?

Для этого мы можем воспользоваться методом [`transforms.RandomApply`](https://pytorch.org/vision/main/generated/torchvision.transforms.RandomApply.html?highlight=randomapply#torchvision.transforms.RandomApply), который на вход принимает список аугментаций и вероятность `p`, с которой каждая аугментация будет применена или не применена.

In [ ]:
transform = transforms.RandomApply(
    transforms=[
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.RandomPerspective(distortion_scale=0.5),
        transforms.ColorJitter(brightness=0.5, hue=0.3),
    ],
    p=0.9,
)

plot_augmented_img(transform, input_img)

##### <font size="4">Random Choice</font>

В других случаях может быть полезен метод [`transforms.RandomChoice`](https://pytorch.org/vision/main/generated/torchvision.transforms.RandomChoice.html?highlight=randomchoice#torchvision.transforms.RandomChoice), который на вход принимает список аугментаций `transforms`, выбирает из него **одну** случайную аугментацию и применяет ее к изображению. Необязательным параметром является список вероятностей `p`, который укзаывает, с какой вероятностью каждая из аугментаций может быть выбрана из списка (по умолчанию каждая может быть выбрана равновероятно).

In [ ]:
transform = transforms.RandomChoice(
    transforms=[
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.RandomPerspective(distortion_scale=0.5, p=1.0),
        transforms.ColorJitter(brightness=0.5, hue=0.3),
    ],
    p=[0.2, 0.4, 0.6],
)

plot_augmented_img(transform, input_img)
plot_augmented_img(transform, input_img)
plot_augmented_img(transform, input_img)

#### Пример создания собственной аугментации

Иногда может оказаться, что среди широкого спектра реализованных аугментаций нет такой, какую вы хотели бы применить к своим данным. В таком случае ее можно описать в виде класса и использовать наравне с реализованными в библиотеке.

Главное, что необходимо описать при создании класса — метод `__call__`. Он должен принимать изображение (оно может быть представлено в формате `PIL.Image`, `np.array` или `torch.Tensor`), делать с ним интересующие нас видоизменения и возвращать измененное изображение.

Рассмотрим пример добавления на изображение [шума "соль и перец"](https://ru.wikipedia.org/wiki/Salt_and_pepper). Наш метод аугментации будет и принимать на вход, и возвращать `PIL.Image`. 



In [ ]:
from PIL import Image


class SaltAndPepperNoise:
    """
    Add a "salt and pepper" noise to the PIL image
    __call__ method returns PIL Image with noise
    """

    def __init__(self, p=0.01):
        self.p = p  # noise level

    def __call__(self, pil_image):
        np_image = np.array(pil_image)

        # create random mask for "salt" and "pepper" pixels
        salt_ind = np.random.choice(
            a=[True, False], size=np_image.shape[:2], p=[self.p, 1 - self.p]
        )
        pepper_ind = np.random.choice(
            a=[True, False], size=np_image.shape[:2], p=[self.p, 1 - self.p]
        )

        # add "salt" and "pepper"
        np_image[salt_ind] = 255
        np_image[pepper_ind] = 0

        return Image.fromarray(np_image)

In [ ]:
transform = SaltAndPepperNoise(p=0.03)

plot_augmented_img(transform, input_img)

#### Аугментация внутри `Dataset`

Возьмем папку с картинками.

In [ ]:
import os
from zipfile import ZipFile
from IPython.display import clear_output

os.chdir("/content")
# download files
!wget --no-check-certificate 'https://edunet.kea.su/repo/EduNet-web_dependencies/L11/for_transforms.Compose.zip' -O data.zip
with ZipFile(
    "data.zip", "r"
) as folder:  # Create a ZipFile Object and load sample.zip in it
    folder.extractall()  # Extract all the contents of zip file in current directory
clear_output()

In [ ]:
os.chdir("/content/for_transforms.Compose")
img_list = os.listdir()
print(img_list)

Напишем класс `Dataset`

In [ ]:
from torch.utils.data import Dataset


class AugmentationDataset(Dataset):
    def __init__(self, img_list, transforms=None):
        self.img_list = img_list
        self.transforms = transforms

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, i):
        img = plt.imread(self.img_list[i])
        img = Image.fromarray(img).convert("RGB")
        img = np.array(img).astype(np.uint8)

        if self.transforms is not None:
            img = self.transforms(img)
        return img

Напишем вспомогательную функцию для отображения картинок. Напомним, что в PyTorch размерность каналов идет в первом, а не в последнем измерении тензора, описывающего картинку: `Channels x Height x Width`. Для отображения при помощи Matplotlib необходимо перевести массив в формат `Height x Width x Channels`.

In [ ]:
def show_img(img):
    plt.figure(figsize=(40, 38))
    img_np = img.numpy()
    plt.imshow(np.transpose(img_np, (1, 2, 0)))  # [CxHxW] -> [HxWxC] for imshow
    plt.show()

Создадим `list` с аугментациями, которые мы хотим применить. Чтобы загрузить аугментации в PyTorch, нам необходимо эти картинки преобразовать в тензоры. Для этого воспользуемся стандартным преобразованием `transforms.ToTensor()`

In [ ]:
tensor_transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((164, 164)),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.RandomPerspective(distortion_scale=0.5),
        transforms.ToTensor(),
    ]
)

Теперь обернем все в `DataLoader` и отобразим

In [ ]:
from torch.utils.data import DataLoader
import torchvision

Augmentation_dataloader = DataLoader(
    AugmentationDataset(img_list, tensor_transform), batch_size=8, shuffle=True
)

data = iter(Augmentation_dataloader)
show_img(torchvision.utils.make_grid(next(data)))

#### Нестандартные способы аугментации

Существуют и более сложные способы аугментации. Ниже приведена пара примеров таких способов.

##### <font size="4">Mixup</font>

Mixup — это "смешение" признаков двух объектов в определенных пропорциях. Mixup можно представить с помощью простого уравнения:

$\text{New image} = \alpha * \text{image}_1 + (1-\alpha) * \text{image}_2$

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/mixup_augmentation_scheme.png" width="700">

Подробнее в статьях:

[mixup: Beyond Empirical Risk Minimization](https://arxiv.org/abs/1710.09412)

[On Mixup Training](https://proceedings.neurips.cc/paper/2019/file/36ad8b5f42db492827016448975cc22d-Paper.pdf)



##### <font size="4">Аугментация при помощи генерации данных</font>

В ряде случаев возможно расширение набора данных путем синтеза новых данных.

Например, при создании системы распознавания текста, можно генерировать новые образцы путем набора распознаваемых фраз или символов различными шрифтами на различных фонах и с добавлением каких-либо шумов и искажений.

В ряде областей для синтеза новых образов могут создаваться 3D-модели распознаваемых объектов. Например, в работе от Microsoft [Fake It Till You Make It: Face analysis in the wild using synthetic data alone](https://microsoft.github.io/FaceSynthetics/) анализ лиц людей производился на синтетических 3D-моделях лиц. Датасет доступен на [GitHub](https://github.com/microsoft/FaceSynthetics).

Также созданием новых образов, похожих на имеющиеся в датасете, можно заниматься при помощи генеративных моделей. Примером генеративных моделей является [GAN](https://ru.wikipedia.org/wiki/Генеративно-состязательная_сеть) (Generative Adversarial Network). Мы познакомимся с такими моделями в одной из следующих лекций.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L11/augmentation_using_data_synthesis.png" width="600">

#### Аугментация в реальных задачах

Кроме методов, реализованных в PyTorch, существуют и специализированные библиотеки для аугментации изображений, в которых реализованы дополнительные возможности (например, наложение теней, бликов или пятен воды на изображение).

Например:
- [Albumentations](https://albumentations.ai)
- [imgaug](https://imgaug.readthedocs.io/en/latest/index.html)
- [AugLy](https://github.com/facebookresearch/AugLy)

**Важно: при выборе методов аугментации имеет смысл использовать только те, которые будут в реальной жизни.**

Например, нет смысла:
- делать перевод изображения в черно-белое, если предполагается, что весь входящий поток будет цветным
- отражать человека вверх ногами, если мы не предполагаем его таким распознавать

### Аудио

Рассмотрим несколько примеров аугментаций аудио. С полным списком можно ознакомиться здесь: [[git] audiomentations](https://github.com/iver56/audiomentations).

Импортируем библиотеку и посмотрим на пример

In [ ]:
os.chdir("/content")

!pip install audiomentations

!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L11/audio_example.wav
clear_output()

In [ ]:
from IPython.display import Audio

# Get input audio
input_audio = "/content/audio_example.wav"

display(Audio(input_audio))

In [ ]:
import librosa

data, sr = librosa.load("/content/audio_example.wav")  # sr - sampling rate

#### Background Noise


In [ ]:
from audiomentations import AddGaussianSNR

augment = AddGaussianSNR(min_snr_in_db=3, max_snr_in_db=7, p=1)

# Augment/transform the audio data
augmented_data = augment(samples=data, sample_rate=sr)

display(Audio(augmented_data, rate=sr))

Сравним волновые картины и спектрограммы

In [ ]:
from scipy.signal import spectrogram


def produce_plots(input_audio_arr, aug_audio, sr):
    f, t, Sxx_in = spectrogram(
        input_audio_arr, fs=sr
    )  # Compute spectrogram for the original signal (f - frequency, t - time)
    f, t, Sxx_aug = spectrogram(aug_audio, fs=sr)

    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20, 5))

    ax[0, 0].plot(input_audio_arr)
    ax[0, 0].set_xlim(0, len(input_audio_arr))
    ax[0, 0].set_xticks([])
    ax[0, 0].set_title("Original audio")

    ax[0, 1].plot(aug_audio)
    ax[0, 1].set_xlim(0, len(input_audio_arr))
    ax[0, 1].set_xticks([])
    ax[0, 1].set_title("Augmented  audio")

    ax[1, 0].imshow(
        np.log(Sxx_in),
        extent=[t.min(), t.max(), f.min(), f.max()],
        aspect="auto",
        cmap="inferno",
    )
    ax[1, 0].set_ylabel("Frequecny, Hz")
    ax[1, 0].set_xlabel("Time,s")

    ax[1, 1].imshow(
        np.log(Sxx_aug, where=Sxx_aug > 0),
        extent=[t.min(), t.max(), f.min(), f.max()],
        aspect="auto",
        cmap="inferno",
    )
    ax[1, 1].set_ylabel("Frequecny, Hz")
    ax[1, 1].set_xlabel("Time,s")

    plt.subplots_adjust(hspace=0)
    plt.show()


produce_plots(data, augmented_data, sr)

#### Time Stretch

In [ ]:
from audiomentations import TimeStretch

augment = TimeStretch(min_rate=0.8, max_rate=1.5, p=1)
augmented_data = augment(data, sample_rate=sr)

display(Audio(augmented_data, rate=sr))

In [ ]:
produce_plots(data, augmented_data, sr)

#### Pitch Shift

Изменение тональности:

In [ ]:
from audiomentations import PitchShift

augment = PitchShift(min_semitones=1, max_semitones=12, p=1)
augmented_data = augment(data, sample_rate=sr)

display(Audio(augmented_data, rate=sr))

#### Совмещаем несколько аугментаций вместе

Как и в случае с картинками, мы можем совмещать несколько аугментаций вместе

In [ ]:
from audiomentations import Compose, AddGaussianNoise, Shift

augment = Compose(
    [
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=1),
        TimeStretch(min_rate=0.8, max_rate=1.25, p=1),
        PitchShift(min_semitones=-4, max_semitones=4, p=1),
        Shift(min_fraction=-0.5, max_fraction=0.5, p=1),
    ]
)

augmented_data = augment(data, sample_rate=sr)

display(Audio(augmented_data, rate=sr))

Посмотрим на то, что получилось:

In [ ]:
produce_plots(data, augmented_data, sr)

Дополнительные библиотеки для аугментации звука (и волновых функций в целом):
- [torchaudio](https://pytorch.org/tutorials/beginner/audio_preprocessing_tutorial.html)
- [torch-audiomentations](https://github.com/asteroid-team/torch-audiomentations)
- [AugLy](https://github.com/facebookresearch/AugLy)

### Текст

Теперь рассмотрим несколько примеров аугментаций текста. С полным списком можно ознакомиться здесь: [[git] библиотеки](https://github.com/makcedward/nlpaug).

In [ ]:
!pip install -q nlpaug

In [ ]:
# Define input text
text = "Hello, future of AI for Science! How are you today?"
print(f"input text: {text}")

#### Аугментация символов

Заменой на похоже выглядящие:

In [ ]:
import nlpaug.augmenter.char as nac

augment = nac.OcrAug()
augmented_text = augment.augment(text)

print(f"Original:\n{text}")
print(f"Augmented Texts:\n{augmented_text}")

С опечатками, которые учитывают расположение символов на клавиатуре:

In [ ]:
augment = nac.KeyboardAug()
augmented_text = augment.augment(text)

print(f"Original:\n{text}")
print(f"Augmented Texts:\n{augmented_text}")

#### Аугментация слов

С орфографическими ошибками:

In [ ]:
import nlpaug.augmenter.word as naw

augment = naw.SpellingAug()
augmented_text = augment.augment(text, n=3)

print(f"Original:\n{text}")
print(f"Augmented Texts:\n{augmented_text}")

С использованием модели для предсказания новых слов в зависимости от контекста:

In [ ]:
!pip install -q transformers

In [ ]:
from IPython.display import clear_output

# model_type: word2vec, glove or fasttext
augment = naw.ContextualWordEmbsAug(model_path="bert-base-uncased", action="insert")
augmented_text = augment.augment(text)

clear_output()
print(f"Original:\n{text}")
print(f"Augmented Texts:\n{augmented_text}")

#### Аугментация предложений

Мы можем перевести текстовые данные на какой-либо язык, а затем перевести их обратно на язык оригинала. Это может помочь сгенерировать текстовые данные с разными словами, сохраняя при этом контекст текстовых данных.

In [ ]:
!pip -q install sacremoses

In [ ]:
back_translation_aug = naw.BackTranslationAug(
    from_model_name="facebook/wmt19-en-de", to_model_name="facebook/wmt19-de-en"
)
augmented_text = back_translation_aug.augment(text)

clear_output()
print(f"Original:\n{text}")
print(f"Augmented Texts:\n{augmented_text}")

*Instance Crossover Augmentation* — составление новых объектов класса из отдельных предложений того же класса. Например, есть два объекта одного класса "положительный отзыв":
- <font color='red'>очень удобное приложение. Мне понравилось им пользоваться</font>
- <font color='green'>класс! Отличный интерфейс</font>

Тогда можно составить новый объект того же класса из их частей:
- <font color='red'>очень удобное приложение!</font> <font color='green'>Отличный интерфейс</font>

Важно: при любых аугментациях текста на уровне предложений есть шанс создать странные и нелогичные объекты, поэтому использовать их следует с особой осторожностью.

Дополнительные  библиотеки для аугментации текста:

- [TextAugment](https://github.com/dsfsi/textaugment)
- [AugLy](https://github.com/facebookresearch/AugLy)

[Обзор методов аугментации текста с примерами](https://amitness.com/2020/05/data-augmentation-for-nlp/)

# Transfer learning

Как обучить нейросеть на своих данных, когда их мало?

Для таких типовых задач, как классификация изображений, можно воспользоваться одной из существующих архитектур (AlexNet, VGG, Inception, ResNet и т.д.) и просто обучить нейросеть на своих данных. Реализации таких сетей с помощью различных фреймворков уже существуют, так что на данном этапе можно использовать одну из них как черный ящик, не вникая в принцип её работы. Например, в PyTorch есть много уже реализованных известных архитектур: [torchvision.models](https://pytorch.org/vision/stable/models.html).

Однако, глубокие нейронные сети требуют больших объемов данных для успешного обучения. И, зачастую, в нашей частной задаче недостаточно данных для того, чтобы хорошо обучить нейросеть с нуля. **Transfer learning** решает эту проблему. По сути мы пытаемся использовать опыт, полученный нейронной сетью при обучении на некоторой задаче $T_1$, чтобы решать схожую задачу $T_2$.

К примеру, transfer learning можно использовать при решении задачи классификации изображений на небольшом наборе данных. Как уже ранее обсуждалось, при обработке изображений свёрточные нейронные сети в первых слоях "реагируют" на некие простые пространственные шаблоны (к примеру, углы), после чего комбинируют их в сложные осмысленные формы (к примеру, глаза или носы). Вся эта информация извлекается из изображения, на её основе создаются сложные представления данных, которые в результате классифицируются линейной моделью. 

Идея заключается в том, что если изначально обучить модель на некоторой сложной и довольно общей задаче, то можно надеяться, что она (как минимум часть ее слоев), в общем случае, будет извлекать важную информацию из изображений, и полученные представления можно будет успешно использовать для классификации линейной моделью.

Таким образом, берем часть модели, которая, по нашему представлению, отвечает за выделение хороших признаков (часто — все слои, кроме последнего) — feature extractor. Присоединяем к этой части один или несколько дополнительных слоёв для решения уже новой задачи. И учим только эти слои. Cлои feature extractor не учим — они "заморожены".

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/transfer_learning_change_classes_scheme.png" width="700">

Понятно, что не все фильтры модели будут использованы максимально эффективно — к примеру, если мы работаем с изображениями, связанными с едой, возможно, не все фильтры на скрытых слоях предобученной на ImageNet модели окажутся полезны для нашей задачи. Почему бы не попробовать **не только обучить новый классификатор, но и дообучить некоторые промежуточные слои**? При использовании этого подхода мы при обучении дополнительно "настраиваем" и промежуточные слои, называется он **fine-tuning**.

При fine-tuning используют меньший learning rate, чем при обучении нейросети с нуля: мы знаем, что по крайней мере часть весов нейросети выполняет свою задачу хорошо, и не хотим испортить это быстрыми изменениями. 

Кроме этого, можно делать комбинации этих методов: сначала учить только последние добавленные нами слои сети, затем учить еще и самые близкие к ним, и после этого учить уже все веса нейросети вместе. То есть мы можем определить свою **стратегию fine-tuning**. 

Иногда fine-tuning считается синонимом Transfer learning, в этом случае часть от предтренированной сети называют **backbone** ("позвоночник"), а добавленную часть — **head** ("голова").

 

## Порядок действий при transfer learning

Последовательно рассмотрим шаги, необходимые для реализации подхода transfer learning.

**Шаг 1. Получение предварительно обученной модели**

Первым шагом является выбор предварительно обученной модели, которую мы хотели бы использовать в качестве основы для обучения. Основным предположением является то, что признаки, которые умеет выделять из данных предобученная модель, хорошо подойдут для решения нашей частной задачи. Поэтому эффект от Transfer learning будет тем лучше, чем более схожими будут **домены** в нашей задаче и в задаче, на которой предварительно обучалась модель.

Для задач обработки изображений очень часто используются модели, предобученные на ImageNet. Такой подход распространен, однако, если ваша задача связана, например, с обработкой снимков клеток под микроскопом, то модель, предобученная на более близком домене (тоже на снимках клеток, пусть и совсем других), может быть лучшим начальным решением. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/transfer_learning_step_1.png" width="600">

**Шаг 2. Заморозка предобученных слоев**

Мы предполагаем, что первые слои модели уже хорошо натренированы выделять какие-то абстрактные признаки из данных. Поэтому мы не хотим "сломать" их, особенно если начнем на этих признаках обучать новые слои, которые инициализируются случайно: на первых шагах обучения ошибка будет большой и мы можем сильно изменить "хорошие" предобученные веса.

Поэтому требуется "заморозить" предобученные веса. На практике заморозка означает **отключение подсчета градиентов**. Таким образом при последующем обучении параметры с отключенным подсчетом градиентов не будут обновляться.



<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/transfer_learning_step_2.png" width="600">

**Шаг 3. Добавление новых обучаемых слоев**

В отличие от начальных слоев, которые выделяют достаточно общие признаки из данных, более близкие к выходу слои предобученной модели сильно специфичны конкретно под ту задачу, на которую она обучалась. Для моделей, предобученных на ImageNet, последний слой заточен конкретно под предсказание 1000 классов из этого набора данных. Кроме этого, последние слои могут не подходить под новую задачу архитектурно: в новой задаче может быть меньше классов, 10 вместо 1000. Поэтому, требуется **заменить последние один или несколько слоев** предобученной модели на новые, подходящие под нашу задачу. При этом, естественно, веса в этих слоях будут инициализированы случайно. Именно эти слои мы и будем обучать на следующем шаге.




<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/transfer_learning_step_3.png" width="600">

**Шаг 4. Обучение новых слоев**

Все, что нам теперь нужно — обучить новые слои на наших данных. При этом замороженные слои используются лишь как **экстрактор высокоуровневых признаков**.
Обучение такой модели существенно ничем не отличается от обучения любой другой модели: используется обучающая и валидационная выборка, контролируется изменение функции потерь и функционала качества.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/transfer_learning_step_4.png" width="600">

**Шаг 5. Тонкая настройка модели (fine-tuning)**

После того, как мы обучили новые слои модели, и они уже как-то решают задачу, мы можем разморозить ранее замороженные веса, чтобы **тонко настроить** их под нашу задачу, в надежде, что это позволит еще немного повысить качество.

Нужно быть осторожным на этом этапе, использовать learning rate на порядок или два меньший, чем при основном обучении, и одновременно с этим следить за возникновением переобучения. Переобучение при fine-tuning может возникать из-за того, что мы резко увеличиваем количество настраиваемых параметров модели, но при этом наш датасет остается небольшим, и мощная модель может начать заучивать обучающие данные.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/transfer_learning_step_5.png" width="600">

## Практический пример transfer learning

Давайте рассмотрим пример практической реализации такого подхода ([код переработан из этой статьи](https://learnopencv.com/image-classification-using-transfer-learning-in-pytorch/)).

Загрузим датасет EuroSAT и удалим из него 90% файлов. EuroSAT — датасет для классификации спутниковых снимков по типам местности: лес, река, жилая застройка и т. п.

In [ ]:
import random
import torch
import numpy as np

def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(42)

In [ ]:
import os
from IPython.display import clear_output
from random import sample

!wget -N https://edunet.kea.su/repo/EduNet-web_dependencies/L11/EuroSAT.zip # http://madm.dfki.de/files/sentinel/EuroSAT.zip
!unzip -n EuroSAT.zip

os.chdir("/content")
path = "/content/2750/"

for folder in os.listdir(path):
    files = os.listdir(path + folder)
    for file in sample(files, int(len(files) * 0.9)):
        os.remove(path + folder + "/" + file)

clear_output()

Определим аугментации. Для примера будем использовать родные аугментации из библиотеки Torchvision

In [ ]:
from torchvision import transforms

# Applying Transforms to the Data
img_transforms = {
    "train": transforms.Compose(
        [
            transforms.Resize(size=224),  # as in ImageNet
            transforms.RandomRotation(degrees=15),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
        ]
    ),
    # No augmentations on valid data!
    "valid": transforms.Compose(
        [
            transforms.Resize(size=224),
            transforms.ToTensor(),
        ]
    ),
    # No augmentations on test data!
    "test": transforms.Compose(
        [
            transforms.Resize(size=224),
            transforms.ToTensor(),
        ]
    ),
}

Создадим `datasets`

In [ ]:
from torchvision import datasets
from copy import deepcopy

dataset = datasets.ImageFolder(root=path)
# split to train/valid/test
train_set, valid_set, test_set = torch.utils.data.random_split(
    dataset, [int(len(dataset) * 0.8), int(len(dataset) * 0.1), int(len(dataset) * 0.1)]
)

train_set.dataset = deepcopy(dataset)
valid_set.dataset = deepcopy(dataset)
test_set.dataset = deepcopy(dataset)

# define augmentations
train_set.dataset.transform = img_transforms["train"]
valid_set.dataset.transform = img_transforms["valid"]
test_set.dataset.transform = img_transforms["test"]

print(f"Train size: {len(train_set)}")
print(f"Valid size: {len(valid_set)}")
print(f"Test size: {len(test_set)}")

In [ ]:
from torch.utils.data import DataLoader

# Batch size
batch_size = 64

# Number of classes
num_classes = len(dataset.classes)

# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}

# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size, valid_data_size = len(train_set), len(valid_set)

# Create iterators for the Data loaded using DataLoader module
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
print("indexes to class: ")
idx_to_class

В наборе данных не так уж и много изображений. При обучении с нуля нейросеть скорее всего не достигнет высокой точности.

### Обучение готовой архитектуры "с нуля"

Загрузим MobileNet v2 без весов и попробуем обучить "с нуля", то есть с весов, инициализированных случайно.

In [ ]:
from torchvision import models

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = models.mobilenet_v2(weights=None)
print(model)

Последний слой MobileNet дает на выходе предсказания для 1000 классов, а в нашем датасете классов всего 10. Поэтому мы должны изменить выход сети так, чтобы он выдавал 10 предсказаний. Поэтому мы заменяем последний слой модели MobileNet слоем с `num_classes` нейронами, равным числу классов в нашем датасете.

То есть мы "сказали" нашей модели распознавать не 1000, а только `num_classes` классов.

In [ ]:
# Change the final layer of MobileNet Model for Transfer Learning
import torch.nn as nn

# change out classes, from 1000 to 10
model.classifier[1] = nn.Linear(1280, num_classes)  
print(model.classifier)

Затем мы определяем функцию потерь и оптимизатор, которые будут использоваться для обучения.

In [ ]:
import torch.optim as optim

# Define Optimizer and Loss Function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
print(optimizer)

Для тренировки и валидации нашей модели напишем отдельную функцию.

In [ ]:
import time


def train_and_validate(model, criterion, optimizer, num_epochs=25, save_state=False):
    """
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)

    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    """

    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(num_epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch + 1, num_epochs))

        # Set to training mode
        model.train()

        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0

        valid_loss = 0.0
        valid_acc = 0.0

        train_correct = 0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()  # Clean existing gradients
            outputs = model(
                inputs
            )  # Forward pass - compute outputs on input data using the model
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagate the gradients
            optimizer.step()  # Update the parameters

            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            # Compute correct predictions
            train_correct += (torch.argmax(outputs, dim=-1) == labels).float().sum()

        # Compute the mean train accuracy
        train_accuracy = 100 * train_correct / (len(train_loader) * batch_size)

        val_correct = 0
        # Validation - No gradient tracking needed
        with torch.no_grad():
            model.eval()  # Set to evaluation mode

            # Validation loop
            for j, (inputs, labels) in enumerate(valid_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(
                    inputs
                )  # Forward pass - compute outputs on input data using the model
                loss = criterion(outputs, labels)  # Compute loss
                valid_loss += loss.item() * inputs.size(
                    0
                )  # Compute the total loss for the batch and add it to valid_loss

                val_correct += (torch.argmax(outputs, dim=-1) == labels).float().sum()

        # Compute mean val accuracy
        val_accuracy = 100 * val_correct / (len(valid_loader) * batch_size)

        # Find average training loss and training accuracy
        avg_train_loss = train_loss / (len(train_loader) * batch_size)

        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss / (len(valid_loader) * batch_size)

        history.append(
            [
                avg_train_loss,
                avg_valid_loss,
                train_accuracy.detach().cpu(),
                val_accuracy.detach().cpu(),
            ]
        )

        epoch_end = time.time()

        print(
            "Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
                epoch + 1,
                avg_train_loss,
                train_accuracy.detach().cpu(),
                avg_valid_loss,
                val_accuracy.detach().cpu(),
                epoch_end - epoch_start,
            )
        )
        # Saving state for fine_tuning (because we may overfit)
        if save_state:
            os.makedirs("check_points", exist_ok=True)
            torch.save(model.state_dict(), f"check_points/fine_tuning_{epoch + 1}.pth")

    return model, history

Теперь обучим нашу модель:

In [ ]:
num_epochs = 20
trained_model, history = train_and_validate(
    model.to(device), criterion, optimizer, num_epochs
)

torch.save(history, "history_fresh.pt")

Посмотрим на графики:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
fig.suptitle("Fresh learning", fontsize=14)

history = np.array(history)
ax[0].plot(history[:, :2])
ax[0].legend(["Train Loss", "Val Loss"])
ax[1].plot(history[:, 2:])
ax[1].legend(["Train Accuracy", "Val Accuracy"])
ax[0].set_xlabel("Epoch Number")
ax[1].set_xlabel("Epoch Number")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Accuracy")
plt.savefig("loss_curve.png")
ax[0].grid()
ax[1].grid()
plt.show()

Точность на валидационной выборке не превысила 66%. Посмотрим, сможем ли мы добиться большей точности при использовании предобученной модели.

### Обучение готовой архитектуры с предобученными весами

#### Обучение классификационной "головы"

Теперь давайте попробуем использовать transfer learning. 

Загрузим **предобученную на ImageNet** модель MobileNet v2:

In [ ]:
del model
model = models.mobilenet_v2(weights="MobileNet_V2_Weights.DEFAULT")

В данном случае мы не дообучаем скрытые слои нашей модели, поэтому отключаем подсчёт градиентов ("**замораживаем**" параметры).

In [ ]:
# Freeze model parameters
for param in model.parameters():
    param.requires_grad = False

Нам снова нужно изменить выход сети так, чтобы он выдавал 10 классов вместо 1000.

Мы могли бы изменить количество выходов сети, просто подменив последний линейный слой, как в примере обучения с нуля:

`model.classifier[1] = nn.Linear(1280, num_classes)`

Но нужно понимать, что **мы не ограничены архитектурой готовой сети**, и можем как подменять слои, так и добавлять новые. Поэтому в целях демонстрации мы заменим выходной слой исходной сети на два слоя: первый мы добавим "подменой" модуля, а затем добавим активацию и новый выходной слой с `num_classes` выходами с помощью метода `add_module()` класса `Sequential`.

Когда мы подменяем или добавляем слои, по умолчанию подсчет градиентов на них будет включен, и таким образом мы добьемся, что **учиться будут только новые слои**, веса которых инициализируются случайно.


In [ ]:
# Change the final layers of MobileNet Model for Transfer Learning

model.classifier[1] = nn.Linear(1280, 500)  # replace last module to our custom, e.g. with 500 neurons
model.classifier.add_module("2", nn.ReLU())  # add activation
model.classifier.add_module("3", nn.Linear(500, num_classes))  # add new output layer with 10  out classes

print(model.classifier)

In [ ]:
# Define Optimizer and Loss Function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

In [ ]:
num_epochs = 20
trained_model, history = train_and_validate(
    model.to(device), criterion, optimizer, num_epochs
)

torch.save(history, "history_transfer_learning.pt")

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
fig.suptitle("Transfer learning", fontsize=14)

history = np.array(history)
ax[0].plot(history[:, :2])
ax[0].legend(["Train Loss", "Val Loss"])
ax[1].plot(history[:, 2:])
ax[1].legend(["Train Accuracy", "Val Accuracy"])
ax[0].set_xlabel("Epoch Number")
ax[1].set_xlabel("Epoch Number")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Accuracy")
plt.savefig("loss_curve.png")
ax[0].grid()
ax[1].grid()
plt.show()

Сравним между собой обучение с нуля и обучение с предобученными весами.

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
fig.suptitle("Fresh Learning (FL) vs Transfer Learning (TL)", fontsize=14)

history_fresh = np.array(torch.load("history_fresh.pt"))
history_transfer_learning = np.array(torch.load("history_transfer_learning.pt"))

ax[0].plot(history_fresh[:, :2], linestyle='--')
ax[0].set_prop_cycle('color', ['tab:blue', 'tab:orange'])
ax[0].plot(history_transfer_learning[:, :2])
ax[0].legend(["Train Loss (FL)", "Val Loss (FL)", "Train Loss (TL)", "Val Loss (TL)"])

ax[1].plot(history_fresh[:, 2:], linestyle='--')
ax[1].set_prop_cycle('color', ['tab:blue', 'tab:orange'])
ax[1].plot(history_transfer_learning[:, 2:])
ax[1].legend(
    [
        "Train Accuracy (FL)",
        "Val Accuracy (FL)",
        "Train Accuracy (TL)",
        "Val Accuracy (TL)",
    ]
)
ax[0].set_xlabel("Epoch Number")
ax[1].set_xlabel("Epoch Number")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Accuracy")
plt.savefig("loss_curve.png")
ax[0].grid()
ax[1].grid()
plt.show()

Начная с предобученных весов, процесс обучения идет более плавно и модель выдает бо́льшую точность. На валидационной выборке мы получили точность около 74%.

#### Дообучение всех слоев (Fine-tuning)

Посмотрим, сможем ли мы еще немного повысить точность путем тонкой донастройки всех всесов сети.

Проведём процедуру **fine-tuning**. 
В предыдущем варианте с transfer learning обучался только последний слой, добавленный вручную. Давайте проверим это, выведя те слои, в которых включён градиент.

In [ ]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

Мы оставим дообученную голову нейронной сети и продолжим **обучение всей сети с уменьшением темпа обучения**. 

**Разморозим** параметры. `criterion` остаётся тот же, в `optimizer` уменьшим параметр `lr` на порядок.

In [ ]:
# Unfreeze model parameters
for param in model.parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=3e-5)

Пройдём дополнительные 20 эпох и построим графики.

In [ ]:
num_epochs = 20
trained_model, history = train_and_validate(
    model.to(device), criterion, optimizer, num_epochs, save_state=True
)

torch.save(history, "history_finetuning.pt")

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(16, 5))
fig.suptitle("Transfer Learning (TL) AND Finetuning (FT)", fontsize=14)

history_transfer_learning = np.array(torch.load("history_transfer_learning.pt"))
history_finetuning = np.array(torch.load("history_finetuning.pt"))

train_val_loss = np.concatenate(
    (history_transfer_learning[:, :2], history_finetuning[:, :2]), axis=0
)
ax[0].plot(train_val_loss)
ax[0].vlines(19, -0.1, 2.1, color="tab:green", linewidth=2, linestyle="--")
ax[0].legend(["Train Loss", "Val Loss", "TL/FT boundary"])

train_val_acc = np.concatenate(
    (history_transfer_learning[:, 2:], history_finetuning[:, 2:]), axis=0
)

ax[1].plot(train_val_acc)
ax[1].vlines(19, -5, 105, color="tab:green", linewidth=2, linestyle="--")
ax[1].legend(["Train Accuracy", "Val Accuracy", "TL/FT boundary"])

ax[0].set_xlabel("Epoch Number")
ax[1].set_xlabel("Epoch Number")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Accuracy")
plt.savefig("loss_curve.png")
ax[0].grid()
ax[1].grid()
plt.show()

Есть ли эффект от **fine-tuning**? После дообучения ещё на 20 эпохах мы наблюдаем следующие эффекты:


*   Loss дополнительно снизился, хотя до fine-tuning он стремился к выходу на плато
*   точность на валидации превысила 80%, то есть мы получили дополнительно около 6% точности.

При fine-tuning модель может быть склонна к переобучению, так как мы обучаем сложную модель с большим числом параметров на небольшом количестве данных. Поэтому мы используем learning rate на порядок меньший, чем при обычном обучении. Для контроля переобучения следует следить за метриками и ошибкой на валидационной выборке.



Лучшее качество на валидационных данных мы получили на 38 эпохе. При fine-tuning мы сохраняли состояния нейросети на каждой эпохе. Возьмём состояние с 38 эпохи как наиболее оптимальное.

In [ ]:
trained_model.load_state_dict(torch.load("check_points/fine_tuning_18.pth"))  # 38 = 20 (TL) + 18 (FT)
trained_model.eval();

Посмотрим на предсказания

In [ ]:
def predict(model, test_img_name, device):
    """
    Function to predict the class of a single test image
    Parameters
        :param model: Model to test
        :param test_img_name: Test image

    """

    transform = img_transforms["test"]
    test_img = torch.tensor(np.asarray(test_img_name))
    test_img = transforms.ToPILImage()(test_img)
    plt.imshow(test_img)

    test_img_tensor = test_img_name.unsqueeze(0).to(device)

    with torch.no_grad():
        model.eval()
        # Model outputs is logits
        out = model(test_img_tensor).to(device)
        probs = torch.softmax(out, dim=1).to(device)
        topk, topclass = probs.topk(3, dim=1)
        for i in range(3):
            print(
                "Predcition",
                i + 1,
                ":",
                idx_to_class[topclass.cpu().numpy()[0][i]],
                ", Score: ",
                round(topk.cpu().numpy()[0][i], 2),
            )

In [ ]:
print("Shoud be %s\n" % idx_to_class[0])
predict(
    trained_model.to(device),
    test_set[np.where([x[1] == 0 for x in test_set])[0][0]][0],
    device,
)

In [ ]:
print("Shoud be %s\n" % idx_to_class[6])
predict(
    trained_model,
    test_set[np.where([x[1] == 6 for x in test_set])[0][0]][0],
    device,
)

In [ ]:
print("Shoud be %s\n" % idx_to_class[8])
predict(
    trained_model,
    test_set[np.where([x[1] == 8 for x in test_set])[0][0]][0],
    device,
)

* Мы увидели, как использовать предварительно обученную модель на 1000 классов ImageNet для нашей задачи на 10 классов.

* Мы сравнили качество **обучения с нуля, transfer learning и fine-tuning** и научились добиваться максимального качества с помощью этих принципов.


На практике не забывайте о характерной **опасности fine-tuning — переобучении**. Используйте **низкий learning rate** и отслеживайте Loss и показатели качества — возможно, вам будет достаточно **небольшого количества эпох**.

# Metric learning

Существуют задачи, где не представляется возможным разбить данные на классы так, чтобы в каждом классе было достаточно много объектов.

Рассмотрим, например, задачу распознавания лиц.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/videoanalytics.png"  width="800">

На вход системе подается фото лица человека. Требуется сопоставить его с другим изображением или изображениями, например, хранящимися в БД, и таким образом идентифицировать человека на фотографии.

На первый взгляд кажется, что это задача классификации.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/classifier_scheme.png"  width="700">

Все изображения одного человека будем считать относящимися к одному классу, и модель будет этот класс предсказывать. 

Для небольшой организации, в которой всего несколько десятков сотрудников такой подход может сработать.
При этом возникнут проблемы:

1. Чтобы обучить такую ​​систему, нам сначала потребуется много (сотни) разных 
изображений каждого сотрудника.

2. Когда человек присоединяется к организации или покидает ее, приходится менять структуру модели и обучать ее заново.


Это практически невозможно для крупных организаций, где набор и увольнение происходит почти каждую неделю. И в принципе невозможно для города масштаба Москвы или Лондона, в котором миллионы жителей и сотни тысяч приезжих.

## Формирование векторов-признаков (embedding)

Поэтому используется другой подход.
Вместо того, чтобы классифицировать изображения, модель учится выделять ключевые признаки и на их основе строить компактный вектор, достаточно точно описывающий лицо.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/face_as_embedding.png"  width="500">

В англоязычной литературе такие вектора признаков называются **embedding**, и мы тоже будем использовать это обозначение.

Может возникнуть вопрос: не потеряем ли мы важную информацию, сжав изображение в несколько сотен чисел? 

Чтобы ответить на него, вспомним, как работает [фоторобот](https://en.wikipedia.org/wiki/Facial_composite). 

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/photorobot.png"  width="700">

Для получения фотореалистичного изображения лица достаточно нескольких ключевых признаков: глаза, волосы, рот, нос...
Каждый из них кодируется максимум несколькими сотнями целых значений. 

Значит, вектора-признака из 128 вещественных чисел будет более чем достаточно.
Правда интерпретировать значения, которые закодирует в него нейросеть, будет не столь просто.

Если нам удастся обучить модель кодировать в embedding признаки, важные для сравнения, то мы сможем сравнивать векторы между собой.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/face_dist.png"  width="700">




Если расстояние между векторами для лиц, которые похожи друг на друга, будут маленькими, а у непохожих, наоборот, большими, то мы сможем экспериментально подобрать порог $d$ и, сравнивая с ним расстояние между двумя векторами, принимать решение: принадлежат ли они одному человеку или нет.

*Можно оценивать не расстояние, а степень схожести similarity. В этом случае неравенства поменяют знак, но логика останется прежней*

Теперь, чтобы идентифицировать человека, требуется только одно изображение его лица. Эмбеддинг  этого изображения можно сравнить с эмбеддингами других лиц из БД, используя  [k-NN](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) или иной метод кластеризации.

<img src ="http://edunet.kea.su/repo/EduNet-content/L11/out/search_in_embedding_space.jpg"  width="700">


Такая модель не учится классифицировать изображение напрямую по какому-либо из выходных классов. Она учится выделять признаки, важные при сравнении.

Такой подход решает обе проблемы, о которых мы говорили выше:
- Для обучения такой сети нам не требуется много экземпляров объектов одного класса, а   достаточно лишь нескольких.
- Но самое большое преимущество в простоте ее обучения в случае появления новых объектов.

## Сиамская сеть (Siamese Network)

Какая архитектура должна быть у модели, генерирующей векторы признаков?

Можно было бы использовать обычную сеть, обученную для задачи классификации, и затем удалить из нее последний(е) слой.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/embedding_from_classifier.png"  width="800">

Активации последнего слоя представляют собой отклики на некие высокоуровневые признаки, потенциально важные для классификации, и их можно интерпретировать как embedding.

In [ ]:
from torchvision.models import alexnet
import torch

face1 = torch.randn((3, 224, 224))
face2 = torch.randn((3, 224, 224))

model = alexnet(weights="AlexNet_Weights.DEFAULT")
# remove classification layer
model.fc = model.classifier[6] = torch.nn.Identity()

# get embeddings
embedding1 = model(face1.unsqueeze(0))
embedding2 = model(face2.unsqueeze(0))

diff = torch.nn.functional.pairwise_distance(embedding1, embedding2)
print("L2 distance: ", diff.item())

Такой подход будет работать.
Однако можно заметно улучшить точность, используя функцию потерь, которая оценивает именно качество сравнения, а не классификации.


Рассмотрим подход, основанный на методологии, описанной в статье [Siamese Neural Networks for One-shot Image Recognition (Koch et al., 2015)](https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf).



<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/siamese_neural_network_scheme.png"  width="800">

<center><em>Используются две копии одной и той же сети, отсюда и название Siamese Networks.</em></center>

Два входных изображения ($x_1$ и $x_2$) проходят через одну и ту же сверточную сеть, на выходе для каждого изображения генерируется вектор признаков фиксированной длины $h_1$ и $h_2$.

Модель обучается генерировать близкие вектора для изображений одного объекта и далекие для разных.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/siamese_neural_network_idea_scheme.png"  width="700">

Оценивая расстояние между двумя векторами признаков, 
которое будет малым для одних и тех же объектов и большим для различных, мы сможем оценить их сходство.

Это центральная идея сиамских сетей.

### Triplet Loss

Какую функцию потерь использовать для обучения такой сети?

Очевидно, loss function должна будет учитывать не один выход, а как минимум два.

Популярной на сегодняшний день является  `Triplet loss`, которой требуется  три embedding вместо двух.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/triplet_loss_scheme.png"  width="700">

Чтобы сгенерировать три эмбеддинга, модель должна получать на вход три изображения. 

Первые два должны относиться к одному и тому же объекту (человеку), а третье — к другому.


Таким образом, триплет состоит из опорного ("якорного" `anchor`), положительного (`positive`) и отрицательного (`negative`) образцов.

Описание в статье [FaceNet: A Unified Embedding for Face Recognition and Clustering](https://arxiv.org/abs/1503.03832)



Сама функция потерь будет выглядеть следующим образом:

$$TripletLoss = \sum_{1}^{N} L_i(x_i^{a},x_i^{p},x_i^{n})$$

$$L_i(x_i^{a},x_i^{p},x_i^{n})=max(0,\left\| f(x_i^{a}) -f(x_i^{p}) \right\|_2^{2} - \left\| f(x_i^{a}) -f(x_i^{n}) \right\|_2^{2} + margin)$$

Где:


$x_i^{a}$ — базовое изображение (anchor),

$x_i^{p}$ — изображение того же объекта (positive),

$x_i^{n}$ — изображение другого объекта (negative),

$f(x)$ — **нормированный** выход модели (embedding) для входа $x$,


$\left\| x \right\|_2$ — это L2 (Euclidean norm), соответственно $\left\| a \right\|_2^{2}$ — это L2 в квадрате,

$margin$ — это константа или минимальный "зазор", на который расстояние до эмбеддинга негативного объекта обязано превосходить расстояние до позитивного (идея такая же, что в SVM Loss) .






<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/triplet_loss_idea_scheme.png"  width="900">


В ходе обучения с  Triplet Loss расстояние между эмбеддингами опорного объекта и позитивного уменьшается, а для отрицательного увеличивается.

Важным дополнением является то, что embedding-и нормируются. В результате нормировки каждый вектор-признак будет иметь единичную длину.

<img src ="https://edunet.kea.su/repo/EduNet-content/L11/out/sphere_distance.png"  width="300">




Теперь мы можем рассматривать embedding-и как точки на n-мерной сфере с радиусом 1. 

Это удобно, так как все расстояния между embedding будут лежать в интервале [0..2], и нам будет проще подобрать порог для сравнения.

Кроме того, можно использовать другие меры расстояния, например, [косинусное расстояние](https://buildwiki.ru/wiki/Cosine_similarity), которое определяется углом между векторами, лежит в интервале [-1 ..1] и соответствует расстоянию между точками на поверхности сферы.

В [статье](https://arxiv.org/abs/1503.03832) авторы минимизируют Евклидово расстояние, но подход будет работать и для других метрик сходства, например, косинусного расстояния.

В PyTorch есть две реализации TripletLoss

[TripletMarginLoss](https://pytorch.org/docs/stable/generated/torch.nn.TripletMarginLoss.html) — минимизирует $L_p$ норму 

In [ ]:
from torch import nn

triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2)
anchor = torch.randn(100, 128, requires_grad=True)
positive = torch.randn(100, 128, requires_grad=True)
negative = torch.randn(100, 128, requires_grad=True)
loss = triplet_loss(anchor, positive, negative)
print(loss)

[TripletMarginWithDistanceLoss](https://pytorch.org/docs/stable/generated/torch.nn.TripletMarginWithDistanceLoss.html#torch.nn.TripletMarginWithDistanceLoss) — позволяет задать произвольную функцию расстояния.

In [ ]:
import torch.nn.functional as F

triplet_loss = nn.TripletMarginWithDistanceLoss(
    margin=1, distance_function=lambda x, y: 1.0 - F.cosine_similarity(x, y)
)
loss = triplet_loss(anchor, positive, negative)
print(loss)

Другие функции потерь для сиамских сетей:

Исторически первой появилась `Contrastive Loss`, о ней подробнее в статье [Dimensionality Reduction by Learning an Invariant Mapping (Hadsell et al., 2005)](https://www.researchgate.net/publication/4246277_Dimensionality_Reduction_by_Learning_an_Invariant_Mapping)


В PyTorch есть реализация 
[CosineEmbeddingLoss](https://pytorch.org/docs/stable/generated/torch.nn.CosineEmbeddingLoss.html), она позволяет обучать модель на парах изображений, минимизировав [косинусное расстояние](https://buildwiki.ru/wiki/Cosine_similarity) между embedding.


## Реализация сиамской сети


### Загрузка данных

Загрузим небольшой фрагмент датасета с лицами. Внутри архива фото лиц сгруппированы по папкам

```
faces/
├── training/
|   ├── s1/
|   |   ├── 1.pgm
|   |   ├ ...
|   |   └── 9.pgm
|   ├ ... (excluding 5...7)
|   └── s40/
|       ├── 1.pgm
|       ├ ...
|       └── 9.pgm
└── testing/
    ├── s5/
    |   ├── 1.pgm
    |   ├ ...
    |   └── 9.pgm
    ├ ...
    └── s7/
        ├── 1.pgm
        ├ ...
        └── 9.pgm
    
```


В каждой папке фото лица одного и того же человека.

In [ ]:
from IPython.display import clear_output

!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L11/small_face_dataset.zip
!unzip small_face_dataset.zip
clear_output()

Чтобы результаты воспроизводились, зафиксируем SEED

In [ ]:
import os
import numpy as np
import random


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(42)

### Dataset for TripletLoss

Для TripletLoss потребуются три изображения: anchor, positive, negative и метод __get_item__ должен возвращать их нам. Первые два должны принадлежать одному человеку, а третье — другому. 



In [ ]:
from torch.utils.data import Dataset
from glob import glob
from PIL import Image


class SiameseNetworkDataset(Dataset):
    def __init__(self, dir=None, transform=None, splitter="/"):
        self.dir = dir
        self.splitter = splitter
        self.transform = transform
        self.files = glob(f"{self.dir}/**/*.pgm", recursive=True)
        self.data = self.build_index()

    def build_index(self):
        index = {}
        for f in self.files:
            id = self.path2id(f)
            if not id in index:
                index[id] = []
            index[id].append(f)
        return index

    def path2id(self, path):
        return path.replace(self.dir, "").split(self.splitter)[0]

    def __getitem__(self, index):
        anchor_path = self.files[index]
        positive_path = self.find_positive(anchor_path)
        negative_path = self.find_negative(anchor_path)

        # Loading the images
        anchor = Image.open(anchor_path)
        positive = Image.open(positive_path)
        negative = Image.open(negative_path)

        if self.transform is not None:  # Apply image transformations
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative = self.transform(negative)

        return anchor, positive, negative

    def find_positive(self, path):
        id = self.path2id(path)
        all_exept_my = self.data[id].copy()
        all_exept_my.remove(path)
        return random.choice(all_exept_my)

    def find_negative(self, path):
        all_exept_my_ids = list(self.data.keys())
        id = self.path2id(path)
        all_exept_my_ids.remove(id)
        selected_id = random.choice(all_exept_my_ids)
        return random.choice(self.data[selected_id])

    def __len__(self):
        return len(self.files)

Выведем несколько изображений, чтобы убедиться, что класс датасета функционирует должным образом.

In [ ]:
from torch.utils.data import DataLoader
import torchvision
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

# Create dataset instance
siamese_dataset = SiameseNetworkDataset(
    "faces/training/",
    transform=transforms.Compose(
        [
            transforms.Resize((105, 105)),
            transforms.ToTensor(),
        ]
    ),
)

# Create dataloader & extract batch of data from it
vis_dataloader = DataLoader(siamese_dataset, batch_size=8, shuffle=True)
dataiter = iter(vis_dataloader)
example_batch = next(dataiter)  # anc, pos, neg

# Show batch contents
concatenated = torch.cat((example_batch[0], example_batch[1], example_batch[2]), 0)
grid = torchvision.utils.make_grid(concatenated)

plt.axis("off")
plt.imshow(grid.permute(1, 2, 0).numpy())
plt.gcf().set_size_inches(20, 60)
plt.show()

В каждом столбце тройка изображений. Первое и второе принадлежат одному человеку, третье — другому.

### Создание модели

Нас устроит любая модель для работы с изображениями. Например, ResNet18. 

Все, что от нас требуется, это:
- заменить последний слой
- отправлять на анализ три изображения вместо одного. Соответственно на выходе тоже будут три вектора признаков (embedding)


Пожалуй, единственный вопрос — это размерность последнего слоя. В промышленных системах распознавания лиц, которые тренируются на датасетах из миллионов изображений, используются embedding размерностью от 128 до 512.

Для демонстрационной задачи нам должно хватить 32 значений. Количество выходов последнего линейного слоя установим равным 32.

In [ ]:
from torchvision.models import resnet18


class SiameseNet(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.model = resnet18(weights=None)
        # Because we use grayscale images reduce input channel count to one
        self.model.conv1 = nn.Conv2d(
            1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
        )
        # Replace ImageNet 1000 class classifier with 64- out linear layer
        self.model.fc = nn.Linear(self.model.fc.in_features, latent_dim)

    def _forward(self, x):
        out = self.model(x)
        # normalize embedding to unit vector
        out = torch.nn.functional.normalize(out)
        return out

    def forward(self, anchor, positive, negative, latent_dim):
        output1 = self._forward(anchor)
        output2 = self._forward(positive)
        output3 = self._forward(negative)

        return output1, output2, output3

### Dataloaders

Загрузчики данных не отличаются от загрузчиков для обычной сети. 
Единственное отличие — это две аугментации, которые добавили к данным:

*   Случайное отражение по вертикали (RandomHorizontalFlip)
*   Размытие (GaussianBlur)


In [ ]:
from torchvision import transforms as img_transf

# Apply augmentations on train data
img_trans_train = img_transf.Compose(
    [
        img_transf.Resize((105, 105)),
        img_transf.RandomHorizontalFlip(),
        img_transf.GaussianBlur(3, sigma=(0.1, 2.0)),
        img_transf.ToTensor(),
    ]
)

img_trans_test = img_transf.Compose(
    [img_transf.Resize((105, 105)), img_transf.ToTensor()]
)

train_dataset = SiameseNetworkDataset("faces/training/", transform=img_trans_train)
val_dataset = SiameseNetworkDataset("faces/testing/", transform=img_trans_test)

batch_size = 300
train_loader = DataLoader(
    train_dataset, num_workers=2, batch_size=batch_size, shuffle=True
)
val_loader = DataLoader(val_dataset, num_workers=2, batch_size=1, shuffle=False)

### Обучение

Отличие от  сетей для классификации в том, что у модели 3 выхода, и все их надо передать в loss. При этом нет меток в явном виде.
Определить, какой embedding относится к позитивному образцу, а какой — к негативному, можно только порядком их следования.

In [ ]:
def train(num_epochs, model, criterion, optimizer, train_loader, latent_dim):
    loss_history = []
    l = []
    model.train()
    for epoch in range(0, num_epochs):
        for i, batch in enumerate(train_loader, 0):
            anc, pos, neg = batch
            output_anc, output_pos, output_neg = model(
                anc.to(device), pos.to(device), neg.to(device), latent_dim
            )
            loss = criterion(output_anc, output_pos, output_neg)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            l.append(loss.item())
        last_epoch_loss = torch.tensor(l[-len(train_loader) : -1]).mean()
        print("Epoch {} with {:.4f} loss".format(epoch, last_epoch_loss))

    return l, last_epoch_loss

При использовании GPU обучение на 5-ти эпохах займет около 15 секунд:

In [ ]:
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
latent_dim = 32
model = SiameseNet(latent_dim).to(device)
criterion = nn.TripletMarginLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 6
l, _ = train(num_epochs, model, criterion, optimizer, train_loader, latent_dim)

Выведем график loss

In [ ]:
plt.plot([i for i in range(len(l))], l)
plt.ylabel("loss")
plt.xlabel("num of epochs")
x = range(0, len(l), len(l) // num_epochs)
labels = range(0, num_epochs)
plt.xticks(x, labels)
plt.grid()
plt.show()

Видно, что он график loss уже вышел на плато, значит, можно проверять результат.

### Проверка

Для начала выведем тройки изображений из проверочного датасета и посмотрим на расстояния для позитивных и негативных пар. Если модель обучилась, расстояния для позитивных пар будут меньше. Другими словами: расстояние между похожими лицами будет маленьким, а между разными будет большим.

P.S. По умолчанию TripletLoss минимизирует Евклидово расстояние.

In [ ]:
# Helper method for visualization
def show(img, text=None):
    img_np = img.numpy()
    plt.axis("off")
    plt.text(75, 120, text, fontweight="bold")
    plt.imshow(np.transpose(img_np, (1, 2, 0)))  # [CxHxW] -> [HxWxC] for imshow
    plt.show()


def plot_imgs(model, test_loader, latent_dim):
    distances_pos = []
    distances_neg = []
    # print(f'latent_dim: {latent_dim}')  # if you want to check latent_dim
    model.eval()
    with torch.inference_mode():
        for i, batch in enumerate(test_loader, 0):
            anc, pos, neg = batch
            output_anc, output_pos, output_neg = model(
                anc.to(device), pos.to(device), neg.to(device), latent_dim
            )
            # compute euc. distance
            distance_pos = F.pairwise_distance(output_anc, output_pos).item()
            distance_neg = F.pairwise_distance(output_anc, output_neg).item()

            distances_pos.append(distance_pos)
            distances_neg.append(distance_neg)

            if not i % 5:
                concatenated = torch.cat((anc, pos, neg))
                result = "OK" if distance_neg - distance_pos > 0 else "BAD"
                show(
                    torchvision.utils.make_grid(concatenated),
                    f"Positive / negative euclidean distances: {distance_pos:.3f} / {distance_neg:.3f} - {result}",
                )

    return distances_pos, distances_neg


distances_pos, distances_neg = plot_imgs(model, val_loader, latent_dim)

Но такая оценка субъективна, давайте посмотрим на распределение расстояний по категориям:

In [ ]:
import seaborn as sns

distances = {"The same person": distances_pos, "Another person": distances_neg}

ax = sns.displot(distances, kde=True, stat="density")
ax.set(xlabel="Pairwise distance")
plt.show()

Видно, что расстояние между двумя фото одного и того же человека в среднем меньше, чем расстояние между фото разных людей.

Если бы мы проектировали систему распознавания лиц, нужно было бы выбрать порог, чтобы сравнивать с ним расстояние и принимать решение о том, верифицировать фото как подлинное или нет.

Соответственно, для нашего игрушечного датасета такой порог следует выбирать ~= 0.75 при условии, что ошибки первого и второго рода для нас равнозначны.

## Оптимизация гиперпараметров

Часто, когда мы пишем и обучаем сети (будь то с нуля или с помощью transfer learning), мы вынуждены угадывать гиперпараметры (lr, betas и т.д). В случае с learning rate нам есть от чего оттолкнуться (маленький lr для transfer learning), но все же такой подход не кажется оптимальным.

Для оптимизации гиперпараметров существуют готовые решения, которые используют различные методы black-box оптимизации. Разберем одну из наиболее популярных библиотек — [**Optuna**](https://optuna.org/).

In [ ]:
!pip install --quiet optuna

Давайте оптимизируем learning rate и latent_dim. Для того, чтобы код не выполнялся очень долго, укажем небольшой диапазон параметров и небольшое число эпох.

In [ ]:
import optuna
from optuna.samplers import RandomSampler


# define function which will optimized
def objective(trial):
    # boundaries for the optimizer's
    lr = trial.suggest_float("lr", 1e-4, 1e-2)
    latent_dim = trial.suggest_int("latent_dim", 8, 64, step=8)

    # create new model(and all parameters) every iteration
    model = SiameseNet(latent_dim).to(device)
    criterion = nn.TripletMarginLoss()
    optimizer = optim.Adam(
        model.parameters(), lr=lr
    )  # learning step regulates by optuna

    # To save time, we will take only 3 epochs
    train_loader = DataLoader(
        train_dataset, num_workers=2, batch_size=batch_size, shuffle=True
    )
    _, last_epoch_loss = train(3, model, criterion, optimizer, train_loader, latent_dim)
    return last_epoch_loss


# Create "exploration"
study = optuna.create_study(
    direction="minimize", study_name="Optimizer", sampler=RandomSampler(42)
)

study.optimize(
    objective, n_trials=10
)  # The more iterations, the higher the chances of catching the most optimal hyperparameters

Как видите, такой упрощенный подбор даже трех параметров модели, занимает много времени.

In [ ]:
# show best params
study.best_params

Давайте посмотрим на историю оптимизации наших параметров:

In [ ]:
optuna.visualization.plot_optimization_history(study)

Что ж, проверим, а станет ли реально лучше

In [ ]:
model = SiameseNet(study.best_params["latent_dim"]).to(device)
criterion = nn.TripletMarginLoss()
optimizer = optim.Adam(
    model.parameters(), lr=study.best_params["lr"]
)  # take lr, which choosen Optuna

num_epochs = 6
train_loader = DataLoader(
    train_dataset, num_workers=2, batch_size=batch_size, shuffle=True
)  # take batch_size", which choosen Optuna
l_optim, _ = train(
    num_epochs,
    model,
    criterion,
    optimizer,
    train_loader,
    study.best_params["latent_dim"],
)  # take latent_dim, which choosen Optuna

In [ ]:
plt.plot([i for i in range(len(l))], l, label="no optimization")
plt.plot([i for i in range(len(l_optim))], l_optim, label="optimal params")
plt.ylabel("loss")
plt.xlabel("num of epochs")
plt.xticks(x, labels)
plt.grid()
plt.legend()
plt.show()

In [ ]:
distances_pos, distances_neg = plot_imgs(
    model, val_loader, study.best_params["latent_dim"]
)

In [ ]:
distances_optim = {"The same person": distances_pos, "Another person": distances_neg}

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

sns.histplot(distances, kde=True, stat="density", alpha=0.5, ax=axes[0])
sns.histplot(distances_optim, kde=True, stat="density", alpha=0.5, ax=axes[1])

axes[0].set(title="No optimization")
axes[1].set(title="Otimization with Optuna")
axes[0].set(xlabel="Pairwise distance")
axes[1].set(xlabel="Pairwise distance")

plt.show()

Распределение расстояний стало лучше, хотя по-прежнему не идеально.

Это объясняется тем, что 10 итераций для подбора гиперпараметров все-таки маловато.

<font size = "6">Заключение</font>


Мы затронули проблемы, которые возникают при обучении на реальных данных.

Одна из основных проблем — малые датасеты. Для того, чтобы обучить нейронную сеть на небольшом датасете, можно использовать:
- аугментации;
- Transfer learning.

Однако необходимо помнить, что ни один из этих методов не защитит от ситуации, когда реальные данные будут сильно отличаться от тренировочных.

В случае когда у нас не только мало данных, но еще и очень большое (возможно, неизвестное) число классов, можно воспользоваться One-shot Learning. В этом случае нейронная сеть обучается не классифицировать изображения, а, наоборот, находить различия между классом и новыми данными. Для этого используются нейронные сети, относящиеся к классу сиамских нейронных сетей.

Также мы разобрали автоматическую оптимизацию гиперпараметров с помощью Optuna и показали, что это эффективный способ сделать нейросеть лучше малой ценой.

<font size = "6">Литература</font>

<font size = "5">Обучение на реальных данных</font>

[How to avoid machine learning pitfalls: a guide for academic researchers (Lones, 2021)](https://arxiv.org/abs/2108.02497)

[Understanding data augmentation for classification: when to warp? (Wong et al., 2016)](https://arxiv.org/abs/1609.08764) 

[Learning from class-imbalanced data: Review of methods and applications (Haixiang et al., 2017)](https://www.sciencedirect.com/science/article/abs/pii/S0957417416307175?via%3Dihub)

<font size = "5">Как решить проблему маленького количества данных?</font>

[Блог-пост о том, как решать проблему малого количества данных.](https://towardsdatascience.com/breaking-the-curse-of-small-datasets-in-machine-learning-part-1-36f28b0c044d)

<font size = "5">Несбалансированные данные</font>

[Imbalanced Data: How to handle Imbalanced Classification Problems](https://medium.com/@cmukesh8688/how-to-handle-imbalanced-classification-problems-4a96f42ae4c4).

[SMOTE explained for noobs - Synthetic Minority Over-sampling TEchnique line by line](https://rikunert.com/SMOTE_explained)

[Блог пост про 8 тактик борьбы с несбалансированными классами в наборе данных машинного обучения](https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/)

[Метрики, разработаные для работы с несбалансированными классами.](https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/#:~:text=Classification%20Accuracy%20is%20Not%20Enough%3A%20More%20Performance%20Measures%20You%20Can%20Use,-By%20Jason%20Brownlee&text=When%20you%20build%20a%20model,This%20is%20the%20classification%20accuracy)


<font size = "5">Transfer Learning</font>

[Image Classification using Transfer Learning in PyTorch](https://learnopencv.com/image-classification-using-transfer-learning-in-pytorch/)

[How To Do Transfer Learning For Computer Vision | PyTorch Tutorial](https://www.youtube.com/watch?v=6nQlxJvcTr0)

[Transfer learning for Computer Vision Tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

[Python Pytorch Tutorials # 2 Transfer Learning : Inference with ImageNet Models](https://www.youtube.com/watch?v=Upw4RaERZic)

[PyTorch - The Basics of Transfer Learning with TorchVision and AlexNet](https://www.youtube.com/watch?v=8etkVC93yU4)


<font size = "5">Augmentation</font>

[A survey on Image Data Augmentation for Deep Learning (Shorten and Khoshgoftaar, 2019)](https://journalofbigdata.springeropen.com/articles/10.1186/s40537-019-0197-0)

[Data augmentation for improving deep learning in image classification problem](https://www.researchgate.net/publication/325920702_Data_augmentation_for_improving_deep_learning_in_image_classification_problem)


<font size = "5">Few-shot learning</font>

[One-Shot Learning with Siamese Networks using Keras](https://towardsdatascience.com/one-shot-learning-with-siamese-networks-using-keras-17f34e75bb3d)

[One-Shot image classification by meta learning](https://medium.com/nerd-for-tech/one-shot-learning-fe1087533585)

[One-Shot Learning (Part 1/2): Definitions and fundamental techniques](https://heartbeat.fritz.ai/one-shot-learning-part-1-2-definitions-and-fundamental-techniques-1df944e5836a)

[One-Shot Learning (Part 2/2): Facial Recognition Using a Siamese Network](https://heartbeat.fritz.ai/one-shot-learning-part-2-2-facial-recognition-using-a-siamese-network-5aee53196255)

[FaceNet: A Unified Embedding for Face Recognition and Clustering (Schroff et al., 2015)](https://arxiv.org/abs/1503.03832)

[One-Shot Learning explained using FaceNet](https://medium.com/intro-to-artificial-intelligence/one-shot-learning-explained-using-facenet-dff5ad52bd38)

[Ищем знакомые лица](https://habr.com/ru/post/317798/)

[Siamese Neural Networks for One-shot Image Recognition (Koch et al., 2015)](https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf)

[Dimensionality Reduction by Learning an Invariant Mapping (Hadsell et al., 2005)](https://www.researchgate.net/publication/4246277_Dimensionality_Reduction_by_Learning_an_Invariant_Mapping)

[Language Models are Few-Shot Learners (Brown et al., 2020)](https://arxiv.org/abs/2005.14165)

<font size = "5">Hyperparameter optimization</font>

[Tuning Hyperparameters with Optuna](https://towardsdatascience.com/tuning-hyperparameters-with-optuna-af342facc549)